In [ ]:
import pandas as pd
import zipfile
import io
import os
import numpy as np

#analise = "depois"


# Ano e mês para análise
ano = 2025         # Substitua pelo ano desejado (tem que ser tipo numérico)
ultimo_mes = '09'   # Substitua pelo mês desejado no formato 'MM' (tem que ser tipo texto)


#########@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@   Configurações de Email   @@@@@####

# Configurações
tipo_email = "outlook"  # ou "gmail"
tipo_destinatario = "pessoal"  # ou "grupo"

# Definir destinatários
if tipo_destinatario == "grupo":
    destinatarios = 'mjandussi@fazenda.rj.gov.br; cbborba@fazenda.rj.gov.br; Daniela.Alves@fazenda.rj.gov.br; Nathalia.Souza@fazenda.rj.gov.br; Gustavo.Mello@fazenda.rj.gov.br'
else:  # pessoal
    destinatarios = 'mjandussi@fazenda.rj.gov.br'


# DIRETÓRIOS

# SEFAZ
### Antigos ###
# diretorio2 = f'C:/Users/mjandussi/OneDrive - SEFAZ-RJ/Documentos/GitHub/ranking_siconfi_excel/Arquivos_{ano}/Msc_{analise}'
# diretorio = f'C:/Users/mjandussi/OneDrive - SEFAZ-RJ/Documentos/GitHub/ranking_siconfi_excel/Arquivos_{ano}'
### ATUAL ###
diretorio = f'C:/Users/mjandussi/Documents/ranking_siconfi_excel/Arquivos_{ano}'

# SAMSUNG
#diretorio = f'D:/GitHub/ranking_siconfi_excel/Arquivos_{ano}'

# MAC
#diretorio = fr'/Users/marcelowalther/Documents/@PYTHON/@Aplicação WEB_Ranking'


# Caminhos dos arquivos locais
zip_file_path = f'msc_{ano}_ate_{ultimo_mes}.zip'# Substitua pelo caminho do seu arquivo ZIP
uploaded_template_stn = f'{ano}_Anexo_II_Portaria_STN_642_Leiaute_MSC.xlsx'  # Substitua pelo caminho do seu arquivo Excel


# Formatar Decimais
pd.set_option('float_format', '{:,.2f}'.format)
# Retirar críticas 
pd.set_option('mode.chained_assignment', None)
# Definir a largura máxima das colunas para None (automático o ajuste)
pd.set_option('display.max_colwidth', None) 


# Função para processar arquivos ZIP
def process_zip(file_path):
    csv_files = {}
    with zipfile.ZipFile(file_path, 'r') as z:
        for file_info in z.infolist():
            if file_info.filename.endswith('.csv'):
                with z.open(file_info.filename) as f:
                    csv_files[file_info.filename] = f.read()
    return csv_files




# Processar arquivos ZIP
csv_files = process_zip(f'{diretorio}/{zip_file_path}')


if not csv_files:
    print("Nenhum arquivo .csv encontrado na pasta compactada.")
elif len(csv_files) != int(ultimo_mes):
    print(f"O número de arquivos no ZIP ({len(csv_files)}) não corresponde ao mês informado ({ultimo_mes}).")
else:
    dataframes = []
    for filename, file_content in csv_files.items():
        with io.BytesIO(file_content) as f:
            # Extrai o mês do nome do arquivo (assumindo que o nome do arquivo segue o padrão 'msc_MM_2024.csv')
            mes_str = filename.split('_')[1]
            
            # Lê apenas a linha de cabeçalho real para obter os nomes das colunas
            df_temp = pd.read_csv(f, sep=';', header=1, nrows=0)
            
            # Cria um dicionário com todos os tipos como 'object', exceto a coluna 'VALOR'
            dtype_dict = {col: 'object' for col in df_temp.columns}
            dtype_dict.pop('VALOR', None)  # Remove 'VALOR' do dicionário se não quiser como 'object'
            
            # Importa o arquivo completo com os dtypes especificados, ignorando a primeira linha
            f.seek(0)  # Reseta o ponteiro do arquivo para o início
            df = pd.read_csv(f, sep=';', header=1, dtype=dtype_dict)
            
            # Adiciona a coluna 'mes' ao DataFrame
            df['mes'] = mes_str
            
            # Adiciona o DataFrame à lista
            dataframes.append(df)
    
    if dataframes:
        # Leitura do PCASP ESTENDIDO
        df_temp = pd.read_excel(f'{diretorio}/{uploaded_template_stn}', nrows=0, sheet_name=f'PcaspEstendido{ano}', header=3)
        dtype_dict = {col: 'object' for col in df_temp.columns}
        pc_estendido = pd.read_excel(f'{diretorio}/{uploaded_template_stn}', sheet_name=f'PcaspEstendido{ano}', header=3, dtype=dtype_dict)

        # Precisamos Confirmar que a Coluna CONTA seja do Tipo texto para fazer o "STR Match"
        pc_estendido['CONTA'] = pc_estendido['CONTA'].astype(str)

        # Combina todos os DataFrames em um único DataFrame
        msc_base = pd.concat(dataframes, ignore_index=True)

        # Criando uma cópia original antes da inversão de saldo das retificadoras
        msc_base_orig = msc_base.copy()

        # Pegar só o primeiro elemento de uma coluna
        msc_base["Grupo_Contas"] = msc_base["CONTA"].str[0]

        # Trocar o sinal
        msc_base['VALOR'] = msc_base.apply(lambda x: x['VALOR'] * -1
            if (x['Grupo_Contas'] == '1' and x['NATUREZA_VALOR'] == 'C' and not x['TIPO_VALOR'] == 'period_change')
            or (x['Grupo_Contas'] == '2' and x['NATUREZA_VALOR'] == 'D' and not x['TIPO_VALOR'] == 'period_change')
            or (x['Grupo_Contas'] == '4' and x['NATUREZA_VALOR'] == 'D' and not x['TIPO_VALOR'] == 'period_change')
            or (x['Grupo_Contas'] == '5' and x['NATUREZA_VALOR'] == 'C' and not x['TIPO_VALOR'] == 'period_change')
            or (x['Grupo_Contas'] == '6' and x['NATUREZA_VALOR'] == 'D' and not x['TIPO_VALOR'] == 'period_change')
            or (x['Grupo_Contas'] == '7' and x['NATUREZA_VALOR'] == 'C' and not x['TIPO_VALOR'] == 'period_change')
            or (x['Grupo_Contas'] == '8' and x['NATUREZA_VALOR'] == 'D' and not x['TIPO_VALOR'] == 'period_change')
            else x['VALOR'], axis=1)

        # Filtrando os DataFrames
        msc_base_e = msc_base.query('TIPO_VALOR == "ending_balance"')
        msc_base_b = msc_base.query('TIPO_VALOR == "beginning_balance"')
        msc_base_p = msc_base.query('TIPO_VALOR == "period_change"')

        # Pegando as bases do último mês
        msc_base_e_ult_mes = msc_base_e.query(f'mes == "{ultimo_mes}"')
        msc_base_b_ult_mes = msc_base_b.query(f'mes == "{ultimo_mes}"')

        # D1_00017
        # Verifica se a condição é maior que zero
        if (msc_base_orig['VALOR'] < 0).any():
            resposta_d1_00017 = 'ERRO'
        else:
            resposta_d1_00017 = 'OK'

        d1_00017_t = msc_base_orig.query('VALOR < 0')

        contagem = d1_00017_t.mes.unique()
        erros = len(contagem)   

        # Cria um DataFrame com a resposta
        d1_00017 = pd.DataFrame([resposta_d1_00017], columns=['Resposta'])
        # Insere novas colunas
        d1_00017.insert(0, 'Dimensão', 'D1_00017')
        d1_00017.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com valores negativos')
        d1_00017.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros}')

        #################################################################################################################
        # D1_00018
        analise = msc_base_orig.groupby(['CONTA', 'mes', 'TIPO_VALOR', 'NATUREZA_VALOR'])['VALOR'].sum().reset_index()

        # Pegar só o primeiro elemento de uma coluna
        analise["Grupo_Contas"] = analise["CONTA"].str[0]

        # Aplicando a fórmula para trocar o sinal das contas retificadoras
        analise['VALOR'] = analise.apply(lambda x: x['VALOR'] * -1
        if (x['Grupo_Contas'] == '1' and x['NATUREZA_VALOR'] == 'C' and not x['TIPO_VALOR'] == 'period_change')
        or (x['Grupo_Contas'] == '2' and x['NATUREZA_VALOR'] == 'D' and not x['TIPO_VALOR'] == 'period_change')
        or (x['Grupo_Contas'] == '4' and x['NATUREZA_VALOR'] == 'D' and not x['TIPO_VALOR'] == 'period_change')
        or (x['Grupo_Contas'] == '5' and x['NATUREZA_VALOR'] == 'C' and not x['TIPO_VALOR'] == 'period_change')
        or (x['Grupo_Contas'] == '6' and x['NATUREZA_VALOR'] == 'D' and not x['TIPO_VALOR'] == 'period_change')
        or (x['Grupo_Contas'] == '7' and x['NATUREZA_VALOR'] == 'C' and not x['TIPO_VALOR'] == 'period_change')
        or (x['Grupo_Contas'] == '8' and x['NATUREZA_VALOR'] == 'D' and not x['TIPO_VALOR'] == 'period_change')
        else x['VALOR'], axis=1)


        # Aplicando a fórmula para trocar o sinal do period_change
        analise['VALOR'] = analise.apply(lambda x: x['VALOR'] * -1
        if (x['Grupo_Contas'] == '1' and x['NATUREZA_VALOR'] == 'C' and x['TIPO_VALOR'] == 'period_change')
        or (x['Grupo_Contas'] == '2' and x['NATUREZA_VALOR'] == 'D' and x['TIPO_VALOR'] == 'period_change')
        or (x['Grupo_Contas'] == '3' and x['NATUREZA_VALOR'] == 'C' and x['TIPO_VALOR'] == 'period_change')
        or (x['Grupo_Contas'] == '4' and x['NATUREZA_VALOR'] == 'D' and x['TIPO_VALOR'] == 'period_change')
        or (x['Grupo_Contas'] == '5' and x['NATUREZA_VALOR'] == 'C' and x['TIPO_VALOR'] == 'period_change')
        or (x['Grupo_Contas'] == '6' and x['NATUREZA_VALOR'] == 'D' and x['TIPO_VALOR'] == 'period_change')
        or (x['Grupo_Contas'] == '7' and x['NATUREZA_VALOR'] == 'C' and x['TIPO_VALOR'] == 'period_change')
        or (x['Grupo_Contas'] == '8' and x['NATUREZA_VALOR'] == 'D' and x['TIPO_VALOR'] == 'period_change')
        else x['VALOR'], axis=1)

        analise_b = analise.query('TIPO_VALOR != "ending_balance"')
        analise_b = analise_b.groupby(['mes','CONTA'])['VALOR'].sum().reset_index()
        analise_b['mes'] = analise_b['mes'].astype(str)
        analise_b['chave'] = analise_b['mes'] + analise_b['CONTA']

        analise_e = analise.query('TIPO_VALOR == "ending_balance"')
        analise_e = analise_e.groupby(['mes', 'CONTA'])['VALOR'].sum().reset_index()
        analise_e['mes'] = analise_e['mes'].astype(str)
        analise_e['chave'] = analise_e['mes'] + analise_e['CONTA']

        analise = analise_b.merge(analise_e, on="chave")
        analise['DIF'] = analise['VALOR_x'] - analise['VALOR_y']

        # Definir um limite muito pequeno para considerar como zero (por conta de quando dá notação científica)
        limite_zero = 1e-2  # Por exemplo, considerando valores menores que X (exemplo, se quiser menor que 0.00001 como 4 zeros)

        # Filtro para valores maiores que zero ou menores que zero, mas não tão próximos de zero quanto o 'limite_zero'
        filtro_valores = (analise['DIF'] > limite_zero) | (analise['DIF'] < -limite_zero)

        d1_00018_t = analise[filtro_valores]
        contagem = d1_00018_t.mes_x.unique()
        erros = len(contagem)

        if filtro_valores.any():
            resposta_d1_00018 = 'ERRO'
        else:
            resposta_d1_00018 = 'OK'

        # Cria um DataFrame com a resposta
        d1_00018 = pd.DataFrame([resposta_d1_00018], columns=['Resposta'])
        # Insere novas colunas
        d1_00018.insert(0, 'Dimensão', 'D1_00018')
        d1_00018.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com diferenças na movimentação (SI+ MOV <> SF)')
        d1_00018.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros}')

        #################################################################################################################
        # D1_00019
        df_temp = pd.read_excel(f'{diretorio}/{uploaded_template_stn}', nrows=0, sheet_name='PO', header=4)
        dtype_dict = {col: 'object' for col in df_temp.columns}
        po_stn = pd.read_excel(f'{diretorio}/{uploaded_template_stn}', sheet_name='PO', header=4, dtype=dtype_dict)

        po_stn['Código'] = po_stn['Código'].astype(int)
        po_stn = po_stn.rename(columns={'Código': 'poder_orgao'})

        codigos_na_msc = msc_base_orig.groupby(['IC1'])['VALOR'].sum().reset_index()
        codigos_na_msc = codigos_na_msc.rename(columns={'IC1': 'poder_orgao'})
        codigos_na_msc['poder_orgao'] = codigos_na_msc['poder_orgao'].astype(int)

        po_verif = codigos_na_msc.merge(po_stn, how="left", on="poder_orgao")

        # Verificar se a coluna 'Nomenclatura' está vazia ou contém NaN
        condicao = po_verif['Nomenclatura'].isna() | (po_verif['Nomenclatura'] == '')

        # Filtrar os dados que atendem à condição
        d1_00019_t = po_verif[condicao]
        erros = d1_00019_t.query('VALOR == 0').value_counts().sum()

        if not d1_00019_t.empty:
            resposta_d1_00019 = 'ERRO'
        else:
            resposta_d1_00019 = 'OK'

        # Cria um DataFrame com a resposta
        d1_00019 = pd.DataFrame([resposta_d1_00019], columns=['Resposta'])
        # Insere novas colunas
        d1_00019.insert(0, 'Dimensão', 'D1_00019')
        d1_00019.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com códigos de Poderes incorretos')
        d1_00019.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros}')



        #################################################################################################################
        # D1_00020
        # Primeiro ordena pelas contas, para ficar as contas "juntas" e depois realizar a diferença entre elas
        msc_consolidada_dif = msc_base_orig.sort_values(by=["CONTA", "mes", "TIPO_VALOR"])
        # Depois agrupa o DF para resumir os dados
        msc_consolidada_dif = msc_consolidada_dif.groupby(['CONTA', 'mes', 'TIPO_VALOR'])['VALOR'].sum().reset_index()
        # Depois excluimos o movimento do período
        msc_dif = msc_consolidada_dif[msc_consolidada_dif['TIPO_VALOR'] != 'period_change']
        # Criamos cuma coluna com a diferença entre linhas
        msc_dif['diferenca_valor'] = msc_dif.groupby(['CONTA'])['VALOR'].diff()
        # Como as diferenças ficaram nas linhas do Saldo inicial, então selecionamos apenas as linhas de beginning_balance
        msc_dif = msc_dif[msc_dif['TIPO_VALOR'] == 'beginning_balance']
        # Depois excluímos as linhas com valores vazios
        msc_dif = msc_dif.dropna()

        # Defina uma pequena tolerância (epsilon)
        epsilon = 1e-2  # Um valor pequeno, como 10^-9
        # Modifique sua query para usar a tolerância
        d1_00020_t = msc_dif[np.abs(msc_dif['diferenca_valor']) > epsilon]
        contagem = d1_00020_t['mes'].unique()
        erros = len(contagem)

        # Ajuste a tolerância para até 3 casas decimais
        tolerancia = 1e-3  
        # Verifica se há algum valor diferente de zero na coluna 'DIF'
        condicao = ((abs(msc_dif['diferenca_valor']) > tolerancia) | (abs(msc_dif['diferenca_valor']) < -tolerancia))

        if condicao.any():
            resposta_d1_00020 = 'ERRO'
        else:
            resposta_d1_00020 = 'OK'

        d1_00020 = pd.DataFrame([resposta_d1_00020], columns=['Resposta'])
        # Insere novas colunas
        d1_00020.insert(0, 'Dimensão', 'D1_00020')
        d1_00020.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com SI <> do SF da MSC do mês anterior')
        d1_00020.insert(3, 'OBS', f'Cada MSC vale 1/12 - Erros: {erros}')


        #################################################################################################################
        # D1_00021
        filtro_1 = pc_estendido[pc_estendido['CONTA'].str.match(r"^(1111|1121|1125|1231|1232)")]
        ativ_pcasp = filtro_1.groupby(['CONTA', 'TÍTULO.1', 'NATUREZA DO SALDO', 'STATUS']).sum().reset_index()

        ativ_msc = msc_base_e[msc_base_e['CONTA'].str.match(r"^(1111|1121|1125|1231|1232)")]
        ativ_msc = ativ_msc.groupby(['CONTA', 'NATUREZA_VALOR', 'mes'])['VALOR'].sum().reset_index()

        ### MERGE
        erro_ativ = ativ_msc.merge(ativ_pcasp, on='CONTA', how="left")

        erro_ativ = erro_ativ[(erro_ativ['VALOR'] != 0)]

        erro_ativ = erro_ativ[['CONTA', 'NATUREZA_VALOR', 'NATUREZA DO SALDO', 'TÍTULO.1', 'VALOR', 'mes']]

        erro_ativ['chave'] = erro_ativ['NATUREZA_VALOR'] + erro_ativ['NATUREZA DO SALDO']

        condicao = erro_ativ.query('chave == "CDevedora" or chave == "DCredora"').value_counts().sum()

        d1_00021_t = erro_ativ.query('chave == "C-Devedora" or chave == "D-Credora"')

        contagem = d1_00021_t.mes.unique()
        erros = len(contagem)   

        # Verifica se a condição é maior que zero
        if condicao > 0:
            resposta_d1_00021 = 'ERRO'
        else:
            resposta_d1_00021 = 'OK'

        # Cria um DataFrame com a resposta
        d1_00021 = pd.DataFrame([resposta_d1_00021], columns=['Resposta'])
        # Insere novas colunas
        d1_00021.insert(0, 'Dimensão', 'D1_00021')
        d1_00021.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com contas dos grupos 1111, 1121, 1125, 1231 e 1232 cujo saldo final está com natureza diferente da padrão do PCASP Estendido')
        d1_00021.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros}')
        #d1_00021.insert(4, 'Nota', f'{nota}')

        #################################################################################################################
        # D1_00022
        # Verifica se a coluna "PO" possui campos vazios
        if msc_base['IC1'].isna().any() or (msc_base['IC1'] == '').any():
            resposta_d1_00022 = 'ERRO'
        else:
            resposta_d1_00022 = 'OK'

        # Criando uma coluna auxiliar para identificar linhas com IC1 nulo ou vazio
        msc_base['IC1_null_or_empty'] = msc_base['IC1'].isna() | (msc_base['IC1'] == '')
        # Usando query para filtrar o DataFrame com base na coluna auxiliar
        d1_00022_t = msc_base.query('IC1_null_or_empty == True')
        # Removendo a coluna auxiliar após o filtro, se desejado
        d1_00022_t = d1_00022_t.drop(columns=['IC1_null_or_empty'])

        contagem = d1_00022_t.mes.unique()
        erros = len(contagem) 
            
        # Cria um DataFrame com a resposta
        d1_00022 = pd.DataFrame([resposta_d1_00022], columns=['Resposta'])
        # Insere novas colunas
        d1_00022.insert(0, 'Dimensão', 'D1_00022')
        d1_00022.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com todos os códigos de poder/órgão informados')
        d1_00022.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros}')

        #################################################################################################################
        # D1_00023
        msc_consolidada_anual_e = msc_base.query('IC1 == "10111" or IC1 == "10112"')
        # Não posso agrupar por conta, pois podemos ter uma conta que de fato não teve movimentação no período e assim o saldo fica igual no mes seguinte
        # Por isso agrupado por Grupo de Contas, pois neste caso não há possibilidades de nada ser movimentado dentro de um grupo de contas
        analise = msc_consolidada_anual_e.groupby(['Grupo_Contas', 'mes'])['VALOR'].sum().reset_index()
        analise['diferenca'] = analise['VALOR'].diff()

        d1_00023_t = analise.query('diferenca == 0')
        contagem = d1_00023_t.mes.unique()
        erros = len(contagem)

        condicao = (analise['diferenca'] == 0)

        if condicao.any():
            resposta_d1_00023 = 'ERRO'
        else:
            resposta_d1_00023 = 'OK'

        # Cria um DataFrame com a resposta
        d1_00023 = pd.DataFrame([resposta_d1_00023], columns=['Resposta'])
        # Insere novas colunas
        d1_00023.insert(0, 'Dimensão', 'D1_00023')
        d1_00023.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade matrizes com dados do poder Executivo repetida em mais de um mês')
        d1_00023.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros}')

        #################################################################################################################
        # D1_00024
        msc_consolidada_anual_e = msc_base.query('IC1 == "20211" or IC1 == "20212"')
        # Não posso agrupar por conta, pois podemos ter uma conta que de fato não teve movimentação no período e assim o saldo fica igual no mes seguinte
        # Por isso agrupado por Grupo de Contas, pois neste caso não há possibilidades de nada ser movimentado dentro de um grupo de contas
        analise = msc_consolidada_anual_e.groupby(['Grupo_Contas', 'mes'])['VALOR'].sum().reset_index()
        analise['diferenca'] = analise['VALOR'].diff()

        d1_00024_t = analise.query('diferenca == 0')
        contagem = d1_00024_t.mes.unique()
        erros = len(contagem)

        condicao = (analise['diferenca'] == 0)

        if condicao.any():
            resposta_d1_00024 = 'ERRO'
        else:
            resposta_d1_00024 = 'OK'

        # Cria um DataFrame com a resposta
        d1_00024 = pd.DataFrame([resposta_d1_00024], columns=['Resposta'])
        # Insere novas colunas
        d1_00024.insert(0, 'Dimensão', 'D1_00024')
        d1_00024.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade matrizes com dados do poder Legislativo repetida em mais de um mês')
        d1_00024.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros}')

        #################################################################################################################
        # D1_00025
        filtro_1 = pc_estendido[pc_estendido['CONTA'].str.match(r"^(2111|2112|2113|2114|2121|2122|2123|2124|2125|2126|213|214|215|221|222|223)")]
        pass_pcasp = filtro_1.groupby(['CONTA', 'TÍTULO.1', 'NATUREZA DO SALDO', 'STATUS']).sum().reset_index()

        pass_msc = msc_base_e[msc_base_e['CONTA'].str.match(r"^(2111|2112|2113|2114|2121|2122|2123|2124|2125|2126|213|214|215|221|222|223)")]
        pass_msc = pass_msc.groupby(['CONTA', 'NATUREZA_VALOR', 'mes'])['VALOR'].sum().reset_index()

        ### MERGE
        erro_pass = pass_msc.merge(pass_pcasp, on='CONTA', how="left")

        erro_pass = erro_pass[(erro_pass['VALOR'] != 0)]

        grouped = erro_pass.groupby(['CONTA', 'mes', 'NATUREZA DO SALDO']).sum().reset_index()


        d1_00025_t = grouped.query('(`NATUREZA DO SALDO` == "Credora" and VALOR < 0) or (`NATUREZA DO SALDO` == "Devedora" and VALOR > 0)')

        condicao1 = grouped.query('`NATUREZA DO SALDO` == "Credora" and VALOR < 0').value_counts().sum()
        condicao2 = grouped.query('`NATUREZA DO SALDO` == "Devedora" and VALOR > 0').value_counts().sum()

        contagem = d1_00025_t.mes.unique()
        erros = len(contagem)   
        nota = (100/13) * (13-erros)
        nota = round(nota)/100

        # Verifique se ambas as condições são verdadeiras simultaneamente
        if condicao1 > 0 or condicao2 > 0:
            resposta_d1_00025 = 'ERRO'
        else:
            resposta_d1_00025 = 'OK'

        # Cria um DataFrame com a resposta
        d1_00025 = pd.DataFrame([resposta_d1_00025], columns=['Resposta'])
        # Insere novas colunas
        d1_00025.insert(0, 'Dimensão', 'D1_00025')
        d1_00025.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com contas dos grupos 2111, 2112, 2113,2114, 2121, 2122, 2123, 2124, 2125,2126, 213, 214, 215, 221, 222, 223 cujo saldo final está com natureza diferente da padrão do PCASP Estendido')
        d1_00025.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros}')


        #################################################################################################################
        # D1_00026
        filtro_1 = pc_estendido[pc_estendido['CONTA'].str.match(r"^(2311|2321|232|233|234|235|236)")]
        pl_pcasp = filtro_1.groupby(['CONTA', 'TÍTULO.1', 'NATUREZA DO SALDO', 'STATUS']).sum().reset_index()

        pl_msc = msc_base_e[msc_base_e['CONTA'].str.match(r"^(2311|2321|232|233|234|235|236)")]
        pl_msc = pl_msc.groupby(['CONTA', 'NATUREZA_VALOR', 'mes'])['VALOR'].sum().reset_index()

        ### MERGE
        d1_00026_tp = pl_msc.merge(pl_pcasp, on='CONTA', how="left")

        d1_00026_tp = d1_00026_tp[(d1_00026_tp['VALOR'] != 0)]

        grouped = d1_00026_tp.groupby(['CONTA', 'mes', 'NATUREZA DO SALDO']).sum().reset_index()

        d1_00026_t = grouped.query('`NATUREZA DO SALDO` == "Credora" and VALOR < 0')

        condicao1 = grouped.query('`NATUREZA DO SALDO` == "Credora" and VALOR < 0').value_counts().sum()

        contagem = d1_00026_t.mes.unique()
        erros = len(contagem)   
        nota = (100/13) * (13-erros)
        nota = round(nota)/100

        # Verifica se a condição é maior que zero
        if condicao1 > 0:
            resposta_d1_00026 = 'ERRO'
        else:
            resposta_d1_00026 = 'OK'

        # Cria um DataFrame com a resposta
        d1_00026 = pd.DataFrame([resposta_d1_00026], columns=['Resposta'])
        # Insere novas colunas
        d1_00026.insert(0, 'Dimensão', 'D1_00026')
        d1_00026.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com contas dos grupos 2311, 2312, 232, 233, 234, 235, 236 cujo saldo final está com natureza diferente da padrão do PCASP Estendido')
        d1_00026.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros}')
        #d1_00026.insert(4, 'Nota', f'{nota}')


        #################################################################################################################
        # D1_00027
        condicao = (msc_base['IC2'] == "1") & (msc_base['IC3'].isnull()) & (msc_base['IC4'].isnull())

        # Criando uma coluna auxiliar para identificar linhas que atendem à condição
        msc_base['IC2_IC3_IC4_condition'] = (msc_base['IC2'] == "1") & (msc_base['IC3'].isnull()) & (msc_base['IC4'].isnull())
        # Usando query para filtrar o DataFrame com base na coluna auxiliar
        d1_00027_tl = msc_base.query('IC2_IC3_IC4_condition == True')
        # Removendo a coluna auxiliar após o filtro, se desejado
        d1_00027_tl = d1_00027_tl.drop(columns=['IC2_IC3_IC4_condition'])

        contagem = d1_00027_tl.mes.unique()
        erros = len(contagem)

        d1_00027_t = d1_00027_tl.groupby(['CONTA'])['VALOR'].sum().reset_index()
        d1_00027_t = d1_00027_t.drop(columns=['VALOR'])

        if condicao.any():
            resposta_d1_00027 = 'ERRO'
        else:
            resposta_d1_00027 = 'OK'
            
        # Cria um DataFrame com a resposta
        d1_00027 = pd.DataFrame([resposta_d1_00027], columns=['Resposta'])
        # Insere novas colunas
        d1_00027.insert(0, 'Dimensão', 'D1_00027')
        d1_00027.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade matrizes com contas contábeis com ISF igual a F que não tenham informação complementar de FR')
        d1_00027.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros}')

        #################################################################################################################
        # D1_00028
        analise = msc_base.groupby(['Grupo_Contas', 'mes', 'TIPO_VALOR'])['VALOR'].sum().reset_index()
        # Contas a quantidade de valores por Grupo de contas
        contagem_meses_acuml_msc = analise.query('TIPO_VALOR == "ending_balance"').value_counts().sum()

        contagem_meses = msc_base['mes'].unique()
        contagem_meses = len(contagem_meses)
        # calcula todos os meses acumulados da matriz sobre o total de meses analisados
        condicao = (contagem_meses_acuml_msc/contagem_meses)

        # Convertendo a coluna 'Grupo_Contas' para numérico
        analise['Grupo_Contas'] = pd.to_numeric(analise['Grupo_Contas'], errors='coerce')
        # Filtrando o DataFrame para valores de 'Grupo_Contas' menores que 1 ou maiores que 8
        d1_00028_t = analise[(analise['Grupo_Contas'] < 1) | (analise['Grupo_Contas'] > 8)]

        contagem = d1_00028_t.mes.unique()
        erros = len(contagem)

        # # Verifica se todas as contas esperadas estão presentes (Total do Grupo de COntas deve ser >= a 8)
        if condicao < 8:
            resposta_d1_00028 = 'ERRO'
        else:
            resposta_d1_00028 = 'OK'
            
        d1_00028 = pd.DataFrame([resposta_d1_00028], columns=['Resposta'])
        # Insere novas colunas
        d1_00028.insert(0, 'Dimensão', 'D1_00028')
        d1_00028.insert(2, 'Descrição da Dimensão', 'Verifica se foram enviados valores diferentes de zero em todas as classes de contas da MSC')
        d1_00028.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros}')

        #################################################################################################################
        # D1_00029
        condicao = (
            (msc_base['CONTA'].str.startswith('6211') |
            msc_base['CONTA'].str.startswith('6212') |
            msc_base['CONTA'].str.startswith('6213')) &
            (msc_base['IC2'].isnull()) &
            (msc_base['VALOR'] != 0)
        )
        # Aplicando a condição para filtrar o DataFrame
        d1_00029_t = msc_base[condicao]

        contagem = d1_00029_t.mes.unique()
        erros = len(contagem)   
        nota = (100/13) * (13-erros)
        nota = round(nota)/100

        if condicao.any():
            resposta_d1_00029 = 'ERRO'
        else:
            resposta_d1_00029 = 'OK'
            
        # Cria um DataFrame com a resposta
        d1_00029 = pd.DataFrame([resposta_d1_00029], columns=['Resposta'])
        # Insere novas colunas
        d1_00029.insert(0, 'Dimensão', 'D1_00029')
        d1_00029.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de MSC com contas dos grupos 6211, 6212, 6213 cujos registros não apresentam detalhamento de FR')
        d1_00029.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros}')
        #d1_00029.insert(4, 'Nota', f'{nota}')

        #################################################################################################################
        # D1_00030
        condicao = (
            (msc_base['CONTA'].str.startswith('6211') |
            msc_base['CONTA'].str.startswith('6212') |
            msc_base['CONTA'].str.startswith('6213')) &
            (msc_base['IC4'].isnull()) &
            (msc_base['VALOR'] != 0)
        )
        # Aplicando a condição para filtrar o DataFrame
        d1_00030_t = msc_base[condicao]

        contagem = d1_00030_t.mes.unique()
        erros = len(contagem)   
        nota = (100/13) * (13-erros)
        nota = round(nota)/100

        if condicao.any():
            resposta_d1_00030 = 'ERRO'
        else:
            resposta_d1_00030 = 'OK'
            
        # Cria um DataFrame com a resposta
        d1_00030 = pd.DataFrame([resposta_d1_00030], columns=['Resposta'])
        # Insere novas colunas
        d1_00030.insert(0, 'Dimensão', 'D1_00030')
        d1_00030.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de MSC com contas dos grupos 6211, 6212, 6213 cujos registros não apresentam detalhamento de NR')
        d1_00030.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros}')
        #d1_00030.insert(4, 'Nota', f'{nota}')

        #################################################################################################################
        # D1_00031
        condicao = (
            (msc_base['CONTA'].str.startswith('62213')) &
            (msc_base['IC5'].isnull()) &
            (msc_base['VALOR'] != 0)
        )

        d1_00031_t = msc_base[condicao]

        contagem = d1_00031_t.mes.unique()
        erros = len(contagem)   
        nota = (100/13) * (13-erros)
        nota = round(nota)/100

        if condicao.any():
            resposta_d1_00031 = 'ERRO'
        else:
            resposta_d1_00031 = 'OK'
            
        # Cria um DataFrame com a resposta
        d1_00031 = pd.DataFrame([resposta_d1_00031], columns=['Resposta'])
        # Insere novas colunas
        d1_00031.insert(0, 'Dimensão', 'D1_00031')
        d1_00031.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de MSC com contas dos grupos 62213 cujos registros não apresentam detalhamento de ND')
        d1_00031.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros}')
        #d1_00031.insert(4, 'Nota', f'{nota}')

        #################################################################################################################
        # D1_00032
        condicao = (
            (msc_base['CONTA'].str.startswith('62213')) &
            (msc_base['IC2'].isnull()) &
            (msc_base['VALOR'] != 0)
        )

        d1_00032_t = msc_base[condicao]

        contagem = d1_00032_t.mes.unique()
        erros = len(contagem)   
        nota = (100/13) * (13-erros)
        nota = round(nota)/100

        if condicao.any():
            resposta_d1_00032 = 'ERRO'
        else:
            resposta_d1_00032 = 'OK'
            
        # Cria um DataFrame com a resposta
        d1_00032 = pd.DataFrame([resposta_d1_00032], columns=['Resposta'])
        # Insere novas colunas
        d1_00032.insert(0, 'Dimensão', 'D1_00032')
        d1_00032.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de MSC com contas dos grupos 62213 cujos registros não apresentam a informação complementar de função/subfunção')
        d1_00032.insert(3, 'OBS', f'Cada MSC vale 1/13 - Erros: {erros}')
        #d1_00032.insert(4, 'Nota', f'{nota}'

        #################################################################################################################
        # D1_00033
        condicao = (msc_base['CONTA'].str.startswith('5221') | msc_base['CONTA'].str.startswith('5222') | 
                    msc_base['CONTA'].str.startswith('6221') | msc_base['CONTA'].str.startswith('6222') | 
                    msc_base['CONTA'].str.startswith('6223')) & msc_base['IC3'].isnull()

        d1_00033_t = msc_base[condicao]

        if condicao.any():
            resposta_d1_00033 = 'ERRO'
        else:
            resposta_d1_00033 = 'OK'
            
        # Cria um DataFrame com a resposta
        d1_00033 = pd.DataFrame([resposta_d1_00033], columns=['Resposta'])
        # Insere novas colunas
        d1_00033.insert(0, 'Dimensão', 'D1_00033')
        d1_00033.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de MSC com contas dos grupos 62213 cujos registros não apresentam a informação complementar de FR')
        d1_00033.insert(3, 'OBS', 'Cada MSC vale 1/13')


        #################################################################################################################
        # D1_00034
        filtro_1 = pc_estendido[pc_estendido['CONTA'].str.match(r"^(311|312|313|321|322|323|331|332|333|351|352|353|361|362|363)")]
        vpd_pcasp = filtro_1.groupby(['CONTA', 'TÍTULO.1', 'NATUREZA DO SALDO', 'STATUS']).sum().reset_index()

        vpd_msc = msc_base_e[msc_base_e['CONTA'].str.match(r"^(311|312|313|321|322|323|331|332|333|351|352|353|361|362|363)")]
        vpd_msc = vpd_msc.groupby(['CONTA', 'NATUREZA_VALOR', 'mes'])['VALOR'].sum().reset_index()

        ### MERGE
        erro_vpd = vpd_msc.merge(vpd_pcasp, on='CONTA', how="left")

        erro_vpd = erro_vpd[(erro_vpd['VALOR'] != 0)]

        grouped = erro_vpd.groupby(['CONTA', 'mes', 'NATUREZA DO SALDO']).sum().reset_index()

        d1_00034_t = grouped.query('`NATUREZA DO SALDO` == "Devedora" and VALOR < 0')

        condicao1 = grouped.query('`NATUREZA DO SALDO` == "Devedora" and VALOR < 0').value_counts().sum()

        contagem = d1_00034_t.mes.unique()
        erros = len(contagem)   
        nota = (100/12) * (12-erros)
        nota = round(nota)/100

        # Verifica se a condição é maior que zero
        if condicao1 > 0:
            resposta_d1_00034 = 'ERRO'
        else:
            resposta_d1_00034 = 'OK'

        # Cria um DataFrame com a resposta
        d1_00034 = pd.DataFrame([resposta_d1_00034], columns=['Resposta'])
        # Insere novas colunas
        d1_00034.insert(0, 'Dimensão', 'D1_00034')
        d1_00034.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com contas dos grupos de VPD com natureza diferente do PCASP')
        d1_00034.insert(3, 'OBS', f'Cada MSC vale 1/12 - Erros: {erros}')

        #################################################################################################################
        # D1_00035
        filtro_1 = pc_estendido[pc_estendido['CONTA'].str.match(r"^(411|412|413|421|422|423|424)")]
        vpa_pcasp = filtro_1.groupby(['CONTA', 'TÍTULO.1', 'NATUREZA DO SALDO', 'STATUS']).sum().reset_index()

        vpa_msc = msc_base_e[msc_base_e['CONTA'].str.match(r"^(411|412|413|421|422|423|424)")]
        vpa_msc = vpa_msc.groupby(['CONTA', 'NATUREZA_VALOR', 'mes'])['VALOR'].sum().reset_index()

        ### MERGE
        erro_vpa = vpa_msc.merge(vpa_pcasp, on='CONTA', how="left")

        erro_vpa = erro_vpa[(erro_vpa['VALOR'] != 0)]

        grouped = erro_vpa.groupby(['CONTA', 'mes', 'NATUREZA DO SALDO']).sum().reset_index()

        d1_00035_t = grouped.query('`NATUREZA DO SALDO` == "Credora" and VALOR < 0')

        condicao1 = grouped.query('`NATUREZA DO SALDO` == "Credora" and VALOR < 0').value_counts().sum()

        contagem = d1_00035_t.mes.unique()
        erros = len(contagem)   
        nota = (100/12) * (12-erros)
        nota = round(nota)/100

        # Verifica se a condição é maior que zero
        if condicao1 > 0:
            resposta_d1_00035 = 'ERRO'
        else:
            resposta_d1_00035 = 'OK'

        # Cria um DataFrame com a resposta
        d1_00035 = pd.DataFrame([resposta_d1_00035], columns=['Resposta'])
        # Insere novas colunas
        d1_00035.insert(0, 'Dimensão', 'D1_00035')
        d1_00035.insert(2, 'Descrição da Dimensão', 'Verifica a quantidade de matrizes com contas dos grupos de VPA com natureza diferente do PCASP')
        d1_00035.insert(3, 'OBS', f'Cada MSC vale 1/12 - Erros: {erros}')


        d1_m = pd.concat([d1_00017, d1_00018, d1_00019, d1_00020, d1_00021, d1_00022, d1_00023, d1_00024, d1_00025,
                        d1_00026, d1_00027, d1_00028, d1_00029, d1_00030, d1_00031, d1_00032, d1_00033, d1_00034, d1_00035])


        #################################################################################################################
        #################################################################################################################
        #################################################################################################################
        #################################################################################################################
        #################################################################################################################
        #################################################################################################################
        # D2_00002
        vpd_fundeb = msc_base_e_ult_mes.query('CONTA == "352240000"')
        d2_00002_t = vpd_fundeb.groupby(['mes', 'CONTA'])['VALOR'].sum().reset_index()

        #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
        condicao = d2_00002_t['VALOR'] > 0

        if condicao.any():
            resposta_d2_00002 = 'OK'
        else:
            resposta_d2_00002 = 'ERRO'

        d2_00002 = pd.DataFrame([resposta_d2_00002], columns=['Resposta'])
        d2_00002.insert(0, 'Dimensão', 'D2_00002')
        d2_00002.insert(2, 'Descrição da Dimensão', 'Verifica se foi informado o valor de VPD do FUNDEB')
        d2_00002.insert(3, 'OBS', 'Anexo I-HI da DCA')

        #################################################################################################################
        # D2_00003
        dedu_fundeb = msc_base_e_ult_mes.query('(CONTA == "621310100" or CONTA == "621310200") and IC4 == "17115301"')
        d2_00003_t = dedu_fundeb.groupby(['mes', 'CONTA'])['VALOR'].sum().reset_index()

        #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
        condicao = d2_00003_t['VALOR'] < 0   

        if condicao.any():
            resposta_d2_00003 = 'OK'
        else:
            resposta_d2_00003 = 'ERRO'

        d2_00003 = pd.DataFrame([resposta_d2_00003], columns=['Resposta'])
        d2_00003.insert(0, 'Dimensão', 'D2_00003')
        d2_00003.insert(2, 'Descrição da Dimensão', 'Verifica se foi informado o valor de Deduções de Receitas para formação do FUNDEB')
        d2_00003.insert(3, 'OBS', 'Anexo I-C da DCA')

        #################################################################################################################
        # D2_00004
        rec_fundeb = msc_base_e_ult_mes.query(f'CONTA == "621200000" and IC4 == "17515001"') # OBS: "17580100" Em 2021 e anos anteriores
        d2_00004_t = rec_fundeb.groupby(['mes', 'CONTA'])['VALOR'].sum().reset_index()

        #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
        condicao = d2_00004_t['VALOR'] > 0   

        if condicao.any():
            resposta_d2_00004 = 'OK'
        else:
            resposta_d2_00004 = 'ERRO'

        d2_00004 = pd.DataFrame([resposta_d2_00004], columns=['Resposta'])
        d2_00004.insert(0, 'Dimensão', 'D2_00004')
        d2_00004.insert(2, 'Descrição da Dimensão', 'Verifica se foi informado o valor de Receitas Orçamentárias do FUNDEB')
        d2_00004.insert(3, 'OBS', 'Anexo I-C da DCA')


        #################################################################################################################
        # D2_00005
        if ultimo_mes < "12":
            # Aplicando o filtro na coluna 'CONTA' e expandindo o filtro na coluna 'IC5'
            filtro = (msc_base_e_ult_mes['CONTA'].str.startswith('62213')) & ((msc_base_e_ult_mes['IC5'].str.startswith('319013')) | (msc_base_e_ult_mes['IC5'].str.startswith('319113')))
            # Aplicando o filtro ao DataFrame
            d2_00005_t = msc_base_e_ult_mes[filtro]

            d2_00005_t['dimensao'] = 'D2_00005_Despesas Executadas com Obrigações Patronais'
            d2_00005_t = d2_00005_t.groupby(['dimensao', 'CONTA']).agg({'VALOR': 'sum'})
            d2_00005_t = d2_00005_t.reset_index()

            #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
            condicao = d2_00005_t['VALOR'] > 0  

            if condicao.all():
                resposta_d2_00005 = 'OK'
            else:
                resposta_d2_00005 = 'ERRO'

            d2_00005 = pd.DataFrame([resposta_d2_00005], columns=['Resposta'])
            d2_00005.insert(0, 'Dimensão', 'D2_00005')
            d2_00005.insert(2, 'Descrição da Dimensão', 'Verifica se foi informado o valor de Despesas Orçamentárias com Encargos Patronais')
            d2_00005.insert(3, 'OBS', 'Anexo I-D da DCA')
        else:
            # Aplicando o filtro na coluna 'CONTA' e expandindo o filtro na coluna 'IC5'
            filtro = (((msc_base_e_ult_mes['CONTA'].str.startswith('6221304')) | (msc_base_e_ult_mes['CONTA'].str.startswith('6221305')) 
                       | (msc_base_e_ult_mes['CONTA'].str.startswith('6221306')) | (msc_base_e_ult_mes['CONTA'].str.startswith('6221307'))) 
            & ((msc_base_e_ult_mes['IC5'].str.startswith('319013')) | (msc_base_e_ult_mes['IC5'].str.startswith('319113'))))
            # Aplicando o filtro ao DataFrame
            d2_00005_t = msc_base_e_ult_mes[filtro]

            d2_00005_t['dimensao'] = 'D2_00005_Despesas Executadas com Obrigações Patronais'
            d2_00005_t = d2_00005_t.groupby(['dimensao', 'CONTA']).agg({'VALOR': 'sum'})
            d2_00005_t = d2_00005_t.reset_index()

            #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
            condicao = d2_00005_t['VALOR'] > 0  

            if condicao.all():
                resposta_d2_00005 = 'OK'
            else:
                resposta_d2_00005 = 'ERRO'

            d2_00005 = pd.DataFrame([resposta_d2_00005], columns=['Resposta'])
            d2_00005.insert(0, 'Dimensão', 'D2_00005')
            d2_00005.insert(2, 'Descrição da Dimensão', 'Verifica se foi informado o valor de Despesas Orçamentárias com Encargos Patronais')
            d2_00005.insert(3, 'OBS', 'Anexo I-D da DCA')


        #################################################################################################################
        # D2_00006
        if ultimo_mes < "12":
            # Aplicando o filtro na coluna 'CONTA' e expandindo o filtro na coluna 'IC5'
            filtro = (msc_base_e_ult_mes['CONTA'].str.startswith('62213')) & ((msc_base_e_ult_mes['IC5'].str.startswith('319011')) | (msc_base_e_ult_mes['IC5'].str.startswith('319111')))
            # Aplicando o filtro ao DataFrame
            d2_00006_t = msc_base_e_ult_mes[filtro]

            d2_00006_t['dimensao'] = 'D2_00006_Despesas Orçamentárias com Pessoal'
            d2_00006_t = d2_00006_t.groupby(['dimensao', 'CONTA']).agg({'VALOR': 'sum'})
            d2_00006_t = d2_00006_t.reset_index()

            # Verifica a condição para dar a resposta
            condicao = d2_00006_t['VALOR'] > 0  

            if condicao.all():
                resposta_d2_00006 = 'OK'
            else:
                resposta_d2_00006 = 'ERRO'

            d2_00006 = pd.DataFrame([resposta_d2_00006], columns=['Resposta'])
            d2_00006.insert(0, 'Dimensão', 'D2_00006')
            d2_00006.insert(2, 'Descrição da Dimensão', 'Verifica se foi informado o valor de Despesas Orçamentárias com Pessoal')
            d2_00006.insert(3, 'OBS', 'Anexo I-D da DCA')
        else:
            # Aplicando o filtro na coluna 'CONTA' e expandindo o filtro na coluna 'IC5'
            filtro = (((msc_base_e_ult_mes['CONTA'].str.startswith('6221304')) | (msc_base_e_ult_mes['CONTA'].str.startswith('6221305')) 
                       | (msc_base_e_ult_mes['CONTA'].str.startswith('6221306')) | (msc_base_e_ult_mes['CONTA'].str.startswith('6221307'))) 
            & ((msc_base_e_ult_mes['IC5'].str.startswith('319011')) | (msc_base_e_ult_mes['IC5'].str.startswith('319111'))))
            
            # Aplicando o filtro ao DataFrame
            d2_00006_t = msc_base_e_ult_mes[filtro]

            d2_00006_t['dimensao'] = 'D2_00006_Despesas Orçamentárias com Pessoal'
            d2_00006_t = d2_00006_t.groupby(['dimensao', 'CONTA']).agg({'VALOR': 'sum'})
            d2_00006_t = d2_00006_t.reset_index()

            # Verifica a condição para dar a resposta
            condicao = d2_00006_t['VALOR'] > 0  

            if condicao.all():
                resposta_d2_00006 = 'OK'
            else:
                resposta_d2_00006 = 'ERRO'

            d2_00006 = pd.DataFrame([resposta_d2_00006], columns=['Resposta'])
            d2_00006.insert(0, 'Dimensão', 'D2_00006')
            d2_00006.insert(2, 'Descrição da Dimensão', 'Verifica se foi informado o valor de Despesas Orçamentárias com Pessoal')
            d2_00006.insert(3, 'OBS', 'Anexo I-D da DCA')


        #################################################################################################################
        # D2_00007
        if ultimo_mes < "12":
            # Aplicando o filtro na coluna 'CONTA' e expandindo o filtro na coluna 'IC5'
            dps_custeio = (msc_base_e_ult_mes['CONTA'].str.startswith('62213')) & ((msc_base_e['IC5'].str.startswith('33')))
            # Aplicando o filtro ao DataFrame
            d2_00007_t = msc_base_e_ult_mes[dps_custeio]

            d2_00007_t['dimensao'] = 'D2_00007_Despesas Orçamentárias de Custeio'
            d2_00007_t = d2_00007_t.groupby(['dimensao', 'CONTA']).agg({'VALOR': 'sum'})
            d2_00007_t = d2_00007_t.reset_index()

            # Verifica a condição para dar a resposta
            condicao = d2_00006_t['VALOR'] > 0  

            if condicao.all():
                resposta_d2_00007 = 'OK'
            else:
                resposta_d2_00007 = 'ERRO'

            d2_00007 = pd.DataFrame([resposta_d2_00007], columns=['Resposta'])
            d2_00007.insert(0, 'Dimensão', 'D2_00007')
            d2_00007.insert(2, 'Descrição da Dimensão', 'Verifica se foi informado o valor Despesas Orçamentárias de Custeio')
            d2_00007.insert(3, 'OBS', 'Anexo I-D da DCA')
        else:
            # Aplicando o filtro na coluna 'CONTA' e expandindo o filtro na coluna 'IC5'
            dps_custeio = (((msc_base_e_ult_mes['CONTA'].str.startswith('6221304')) | (msc_base_e_ult_mes['CONTA'].str.startswith('6221305')) 
                       | (msc_base_e_ult_mes['CONTA'].str.startswith('6221306')) | (msc_base_e_ult_mes['CONTA'].str.startswith('6221307'))) 
            & ((msc_base_e_ult_mes['IC5'].str.startswith('33'))))
        
            # Aplicando o filtro ao DataFrame
            d2_00007_t = msc_base_e_ult_mes[dps_custeio]

            d2_00007_t['dimensao'] = 'D2_00007_Despesas Orçamentárias de Custeio'
            d2_00007_t = d2_00007_t.groupby(['dimensao', 'CONTA']).agg({'VALOR': 'sum'})
            d2_00007_t = d2_00007_t.reset_index()

            # Verifica a condição para dar a resposta
            condicao = d2_00006_t['VALOR'] > 0  

            if condicao.all():
                resposta_d2_00007 = 'OK'
            else:
                resposta_d2_00007 = 'ERRO'

            d2_00007 = pd.DataFrame([resposta_d2_00007], columns=['Resposta'])
            d2_00007.insert(0, 'Dimensão', 'D2_00007')
            d2_00007.insert(2, 'Descrição da Dimensão', 'Verifica se foi informado o valor Despesas Orçamentárias de Custeio')
            d2_00007.insert(3, 'OBS', 'Anexo I-D da DCA')

        #################################################################################################################
        # D2_00008
        dps_funcao = (msc_base_e_ult_mes['CONTA'].str.startswith('62213')) & (msc_base_e_ult_mes['TIPO2'].str.startswith('FS')) 
        d2_00008_t = msc_base_e_ult_mes[dps_funcao]
        d2_00008_t['Funcao'] = d2_00008_t['IC2'].str[:2]
        d2_00008_t['dimensao'] = 'D2_00008_Despesas Orçamentárias por Função'
        d2_00008_t = d2_00008_t.groupby(['dimensao', 'Funcao']).agg({'VALOR': 'sum'})
        d2_00008_t = d2_00008_t.reset_index()


        #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
        condicao = d2_00008_t['VALOR'] > 0  

        if condicao.all():
            resposta_d2_00008 = 'OK'
        else:
            resposta_d2_00008 = 'ERRO'

        d2_00008 = pd.DataFrame([resposta_d2_00008], columns=['Resposta'])
        d2_00008.insert(0, 'Dimensão', 'D2_00008')
        d2_00008.insert(2, 'Descrição da Dimensão', 'Verifica se foi informado o valor Despesa Orçamentárias por Função')
        d2_00008.insert(3, 'OBS', 'Anexo I-E da DCA')

        #################################################################################################################
        # D2_00010
        rec_transf = (msc_base_e_ult_mes['CONTA'].str.startswith('6212')) & ((msc_base_e_ult_mes['IC4'].str.startswith('171')) | (msc_base_e_ult_mes['IC4'].str.startswith('172')) | (msc_base_e_ult_mes['IC4'].str.startswith('173')))
        d2_00010_t = msc_base_e_ult_mes[rec_transf]
        d2_00010_t["IC4_resu"] = d2_00010_t["IC4"].str[:3]
        d2_00010_t = d2_00010_t.groupby(['mes', 'CONTA', 'IC4_resu'])['VALOR'].sum().reset_index()

        #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
        condicao = d2_00010_t['VALOR'] > 0  

        if condicao.all():
            resposta_d2_00010 = 'OK'
        else:
            resposta_d2_00010 = 'ERRO'

        d2_00010 = pd.DataFrame([resposta_d2_00010], columns=['Resposta'])
        d2_00010.insert(0, 'Dimensão', 'D2_00010')
        d2_00010.insert(2, 'Descrição da Dimensão', 'Verifica se foi informado o valor das Receitas de Transferências Intergovernamentais')
        d2_00010.insert(3, 'OBS', 'Anexo I-C da DCA')

        #################################################################################################################
        # D2_00011
        rec_tribu = (msc_base_e_ult_mes['CONTA'].str.startswith('6212')) & (msc_base_e_ult_mes['IC4'].str.startswith('11'))
        d2_00011_t = msc_base_e_ult_mes[rec_tribu]
        d2_00011_t = d2_00011_t.groupby(['mes', 'CONTA'])['VALOR'].sum().reset_index()

        #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
        condicao = d2_00011_t['VALOR'] > 0  

        if condicao.any():
            resposta_d2_00011 = 'OK'
        else:
            resposta_d2_00011 = 'ERRO'

        d2_00011 = pd.DataFrame([resposta_d2_00011], columns=['Resposta'])
        d2_00011.insert(0, 'Dimensão', 'D2_00011')
        d2_00011.insert(2, 'Descrição da Dimensão', 'Verifica se foi informado o valor das Receitas Orçamentárias Tributárias')
        d2_00011.insert(3, 'OBS', 'Anexo I-C da DCA')

        #################################################################################################################
        # D2_00012
        receita_e_deducoes = msc_base_e[msc_base_e['CONTA'].str.match(r"^(621)")]
        d2_00012_t = receita_e_deducoes.groupby(['IC4'])['VALOR'].sum().reset_index()

        limiar = 1e-2  # Definir um LIMITE (número decimal 0.01 em notação científica)
        # Substituir valores abaixo do limiar por zero (Isso significa que os valores que estão dentro do limiar tanto acima quanto abaixo de zero serão substituídos por zero)
        d2_00012_t['VALOR'] = d2_00012_t['VALOR'].apply(lambda x: 0 if abs(x) < limiar else x)

        condicao = d2_00012_t['VALOR'] < 0  

        ## Aqui Tem que ser o ANY (e não o ALL), porque precisa apenas que tenha uma NR com valor negativo (Dedução maior que Rec.Bruta)
        if condicao.any():
            resposta_d2_00012 = 'ERRO'
        else:
            resposta_d2_00012 = 'OK'

        d2_00012 = pd.DataFrame([resposta_d2_00012], columns=['Resposta'])
        d2_00012.insert(0, 'Dimensão', 'D2_00012')
        d2_00012.insert(2, 'Descrição da Dimensão', 'Verifica se o valor de deduções de receitas é superior às receitas brutas para cada uma das linhas. As naturezas de receitas começadas por 132 não são analisadas')
        d2_00012.insert(3, 'OBS', 'Anexo I-C da DCA')

        #################################################################################################################
        # D2_00013
        # Para o Curto Prazo precisa fazer a associação por grupo entre o Contas a Receber e o seu Respectivo Ajuste para Perdas
        cred_trib_cp = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1121)")]
        cred_trib_cp['tipo'] = "CP_Credito Tributario"
        clientes_cp = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1122)")]
        clientes_cp['tipo'] = "CP_Clientes"
        emprest_e_financ_cp = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1124)")]
        emprest_e_financ_cp['tipo'] = "CP_Emprestimos e Financiamentos"
        divida_ativa_trib_cp = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1125)")]
        divida_ativa_trib_cp['tipo'] = "CP_Divida Ativa Tributaria"
        divida_ativa_nao_trib_cp = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1126)")]
        divida_ativa_nao_trib_cp['tipo'] = "CP_Divida Ativa Não Tributaria"

        ajuste_perda_cred_trib_cp = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^1129101|1129201|1129301|1129401|1129501")]
        ajuste_perda_cred_trib_cp['tipo'] = "CP_Credito Tributario"
        ajuste_perda_clientes_cp = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^1129102|1129202|1129302|1129402|1129502")]
        ajuste_perda_clientes_cp['tipo'] = "CP_Clientes"
        ajuste_perda_emprest_e_financ_cp = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^1129103|1129203|1129303|1129403|1129503")]
        ajuste_perda_emprest_e_financ_cp['tipo'] = "CP_Emprestimos e Financiamentos"
        ajuste_perda_divida_ativa_trib_cp = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^1129104|1129204|1129304|1129404|1129504")]
        ajuste_perda_divida_ativa_trib_cp['tipo'] = "CP_Divida Ativa Tributaria"
        ajuste_perda_divida_ativa_nao_trib_cp = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^1129105|1129205|1129305|1129405|1129505")]
        ajuste_perda_divida_ativa_nao_trib_cp['tipo'] = "CP_Divida Ativa Não Tributaria"

        # Já para o Longo Prazo, não precisa separar por Grupo de Contas a Receber e sim apenas pelo quinto nivel de consolidação
        cred_lp_extra = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(12111)")]
        cred_lp_extra['tipo'] = "LP_Credito EXTRA"
        cred_lp_intra = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(12112)")]
        cred_lp_intra['tipo'] = "LP_Credito INTRA"
        cred_lp_uniao = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(12113)")]
        cred_lp_uniao['tipo'] = "LP_Credito UNIAO"
        cred_lp_estado = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(12114)")]
        cred_lp_estado['tipo'] = "LP_Credito ESTADO"
        cred_lp_municipio = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(12115)")]
        cred_lp_municipio['tipo'] = "LP_Credito MUNICIPIO"

        d2_00013_t = pd.concat([cred_trib_cp,ajuste_perda_cred_trib_cp,clientes_cp, ajuste_perda_clientes_cp,emprest_e_financ_cp,ajuste_perda_emprest_e_financ_cp,
                                divida_ativa_trib_cp,ajuste_perda_divida_ativa_trib_cp,divida_ativa_nao_trib_cp,ajuste_perda_divida_ativa_nao_trib_cp,
                                cred_lp_extra,cred_lp_intra,cred_lp_uniao,cred_lp_estado,cred_lp_municipio])

        d2_00013_t = d2_00013_t.groupby(['tipo'])['VALOR'].sum().reset_index()


        #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
        # Verifica se existe algum valor negativo na coluna 'valor'
        condicao_negativa = (d2_00013_t['VALOR'] < 0).any()

        if condicao_negativa:
            # Se existe pelo menos um valor negativo, a resposta deve ser 'ERRO'
            resposta_d2_00013 = 'ERRO'
        else:
            # Se não existem valores negativos, a resposta é 'OK'
            resposta_d2_00013 = 'OK'

        d2_00013 = pd.DataFrame([resposta_d2_00013], columns=['Resposta'])
        d2_00013.insert(0, 'Dimensão', 'D2_00013')
        d2_00013.insert(2, 'Descrição da Dimensão', 'Verifica a relação entre o ajuste para perdas dos "Créditos a curto e longo prazo" e suas contas principais. O valor dos ajustes não pode ser superior aos valores das contas originais')
        d2_00013.insert(3, 'OBS', 'Anexo I-AB da DCA')


        #################################################################################################################
        # D2_00014
        # Para o Curto Prazo apenas pelo quinto nivel de consolidação
        outros_cred_cp_extra = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^11311|11321|11331|11341|11351|11361|11371|11381")]
        outros_cred_cp_extra['tipo'] = "CP_Outros Creditos EXTRA"
        outros_cred_cp_intra = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^11312|11322|11332|11342|11352|11362|11372|11382")]
        outros_cred_cp_intra['tipo'] = "CP_Outros Creditos INTRA"
        outros_cred_cp_uniao = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^11313|11323|11333|11343|11353|11363|11373|11383")]
        outros_cred_cp_uniao['tipo'] = "CP_Outros Creditos UNIAO"
        outros_cred_cp_estado = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^11314|11324|11334|11344|11354|11364|11374|11384")]
        outros_cred_cp_estado['tipo'] = "CP_Outros Creditos ESTADO"
        outros_cred_cp_municipio = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^11315|11325|11335|11345|11355|11365|11376|11385")]
        outros_cred_cp_municipio['tipo'] = "CP_Outros Creditos MUNICIPIO"


        # Ajustes do CP
        ajuste_outros_cred_cp_extra = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(11391)")]
        ajuste_outros_cred_cp_extra['tipo'] = "CP_Outros Creditos EXTRA"
        ajuste_outros_cred_cp_intra = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(11392)")]
        ajuste_outros_cred_cp_intra['tipo'] = "CP_Outros Creditos INTRA"
        ajuste_outros_cred_cp_uniao = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(11393)")]
        ajuste_outros_cred_cp_uniao['tipo'] = "CP_Outros Creditos UNIAO"
        ajuste_outros_cred_cp_estado = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(11394)")]
        ajuste_outros_cred_cp_estado['tipo'] = "CP_Outros Creditos ESTADO"
        ajuste_outros_cred_cp_municipio = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(11395)")]
        ajuste_outros_cred_cp_municipio['tipo'] = "CP_Outros Creditos MUNICIPIO"

        # Já para o Longo Prazo, não precisa separar por Grupo de Contas a Receber e sim apenas pelo quinto nivel de consolidação
        outros_cred_lp_extra = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(12121)")]
        outros_cred_lp_extra['tipo'] = "LP_Outros Creditos EXTRA"
        outros_cred_lp_intra = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(12122)")]
        outros_cred_lp_intra['tipo'] = "LP_Outros Creditos INTRA"
        outros_cred_lp_uniao = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(12123)")]
        outros_cred_lp_uniao['tipo'] = "LP_Outros Creditos UNIAO"
        outros_cred_lp_estado = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(12124)")]
        outros_cred_lp_estado['tipo'] = "LP_Outros Creditos ESTADO"
        outros_cred_lp_municipio = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(12125)")]
        outros_cred_lp_municipio['tipo'] = "LP_Outros Creditos MUNICIPIO"

        d2_00014_t = pd.concat([outros_cred_cp_extra, outros_cred_cp_intra, outros_cred_cp_uniao, outros_cred_cp_estado, outros_cred_cp_municipio,
        ajuste_outros_cred_cp_extra,ajuste_outros_cred_cp_intra,ajuste_outros_cred_cp_uniao,ajuste_outros_cred_cp_estado,ajuste_outros_cred_cp_municipio,
                                outros_cred_lp_extra,outros_cred_lp_intra,outros_cred_lp_uniao,outros_cred_lp_estado,outros_cred_lp_municipio])

        d2_00014_t = d2_00014_t.groupby(['tipo'])['VALOR'].sum().reset_index()


        #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
        # Verifica se existe algum valor negativo na coluna 'valor'
        condicao_negativa = (d2_00014_t['VALOR'] < 0).any()

        if condicao_negativa:
            # Se existe pelo menos um valor negativo, a resposta deve ser 'ERRO'
            resposta_d2_00014 = 'ERRO'
        else:
            # Se não existem valores negativos, a resposta é 'OK'
            resposta_d2_00014 = 'OK'

        d2_00014 = pd.DataFrame([resposta_d2_00014], columns=['Resposta'])
        d2_00014.insert(0, 'Dimensão', 'D2_00014')
        d2_00014.insert(2, 'Descrição da Dimensão', 'Verifica a relação entre o ajuste para perdas dos "Créditos a curto e longo prazo" e suas contas principais. O valor dos ajustes não pode ser superior aos valores das contas originais"')
        d2_00014.insert(3, 'OBS', 'Anexo I-AB da DCA')


        #################################################################################################################
        #D2_00015_Bens Móveis
        bens_moveis = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(12311)")]
        bens_moveis['dimensao'] = 'D2_00015_Bens Móveis'
        bens_moveis = bens_moveis.groupby('dimensao').agg({'VALOR': 'sum'})
        d2_00015_t = bens_moveis.reset_index()

        #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
        condicao = d2_00015_t['VALOR'] > 0  

        if condicao.any():
            resposta_d2_00015 = 'OK'
        else:
            resposta_d2_00015 = 'ERRO'

        d2_00015 = pd.DataFrame([resposta_d2_00015], columns=['Resposta'])
        d2_00015.insert(0, 'Dimensão', 'D2_00015')
        d2_00015.insert(2, 'Descrição da Dimensão', 'Verifica a informação de valor patrimonial de bens móveis')
        d2_00015.insert(3, 'OBS', 'Anexo I-AB da DCA')

        #################################################################################################################
        #D2_00016_Depreciação de Bens Móveis
        depr_bens_moveis = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1238101)")]
        depr_bens_moveis['dimensao'] = 'D2_00016_Depreciação de Bens Móveis'
        depr_bens_moveis = depr_bens_moveis.groupby('dimensao').agg({'VALOR': 'sum'})
        d2_00016_t = depr_bens_moveis.reset_index()

        #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
        condicao = d2_00016_t['VALOR'] < 0  #como o sinal esta invertido da depreciação, para verificar precisei colocar menor que zero (para verificar que existe saldo)

        if condicao.any():
            resposta_d2_00016 = 'OK'
        else:
            resposta_d2_00016 = 'ERRO'

        d2_00016 = pd.DataFrame([resposta_d2_00016], columns=['Resposta'])
        d2_00016.insert(0, 'Dimensão', 'D2_00016')
        d2_00016.insert(2, 'Descrição da Dimensão', 'Verifica a Informação de Depreciação acumulada de bens móveis')
        d2_00016.insert(3, 'OBS', 'Anexo I-AB da DCA')

        #################################################################################################################
        #D2_00017_VPD de Depreciação de Bens Móveis e Imóveis
        vpd_depr_bens_moveis = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(33311)")]
        vpd_depr_bens_moveis['dimensao'] = 'D2_00017_VPD de Depreciação de Bens Móveis e Imóveis'
        vpd_depr_bens_moveis = vpd_depr_bens_moveis.filter(items=['dimensao', 'VALOR'])
        d2_00017_t = vpd_depr_bens_moveis.reset_index()
        # Excluir a coluna Index e depois agrupar as contas novamente (só para vizualizar melhor a tabela)
        d2_00017_t.drop(columns=['index'], inplace=True)
        d2_00017_t = d2_00017_t.groupby('dimensao').sum().reset_index()

        #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
        condicao = d2_00017_t['VALOR'] > 0  

        if condicao.any():
            resposta_d2_00017 = 'OK'
        else:
            resposta_d2_00017 = 'ERRO'

        d2_00017 = pd.DataFrame([resposta_d2_00017], columns=['Resposta'])
        d2_00017.insert(0, 'Dimensão', 'D2_00017')
        d2_00017.insert(2, 'Descrição da Dimensão', 'Verifica a informação de variação patrimonial diminutiva de depreciação de bens móveis e imóveis')
        d2_00017.insert(3, 'OBS', 'Anexo I-HI da DCA')

        #################################################################################################################
        # D2_00018_DIF_Entre Depreciação e Bens Móveis
        dif_depr_bens_mov = pd.concat([bens_moveis, depr_bens_moveis])
        # Calcula a diferença
        diff = dif_depr_bens_mov['VALOR'].iloc[-2] - dif_depr_bens_mov['VALOR'].iloc[-1]
        # Cria uma linha no final com este cálculo da diferença usando pd.concat ao invés de append
        nova_linha = pd.DataFrame({'VALOR': [diff]})
        dif_depr_bens_mov = pd.concat([dif_depr_bens_mov, nova_linha], ignore_index=True)
        # Pega então somente esta linha final (da diferença)
        dif_depr_bens_mov = dif_depr_bens_mov.iloc[[len(dif_depr_bens_mov)-1]]
        dif_depr_bens_mov['dimensao'] = 'D2_00018_DIF Depreciação e Bens Móveis'
        d2_00018_t = dif_depr_bens_mov.reset_index()
        # Excluir a coluna Index e depois agrupar as contas novamente (só para vizualizar melhor a tabela)
        d2_00018_t.drop(columns=['index'], inplace=True)
        d2_00018_t = d2_00018_t.groupby('dimensao').sum().reset_index()

        #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
        condicao = d2_00018_t['VALOR'] > 0  

        if condicao.any():
            resposta_d2_00018 = 'OK'
        else:
            resposta_d2_00018 = 'ERRO'

        d2_00018 = pd.DataFrame([resposta_d2_00018], columns=['Resposta'])
        d2_00018.insert(0, 'Dimensão', 'D2_00018')
        d2_00018.insert(2, 'Descrição da Dimensão', 'Avalia se o valor dos bens móveis é maior que sua depreciação acumulada ')
        d2_00018.insert(3, 'OBS', 'Anexo I-AB da DCA')

        #################################################################################################################
        #D2_00019_Bens Imóveis
        bens_imoveis = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(12321)")]
        bens_imoveis['dimensao'] = 'D2_00019_Bens Imóveis'
        bens_imoveis = bens_imoveis.groupby('dimensao').agg({'VALOR': 'sum'})
        d2_00019_t = bens_imoveis.reset_index()

        #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
        condicao = d2_00019_t['VALOR'] > 0  

        if condicao.any():
            resposta_d2_00019 = 'OK'
        else:
            resposta_d2_00019 = 'ERRO'

        d2_00019 = pd.DataFrame([resposta_d2_00019], columns=['Resposta'])
        d2_00019.insert(0, 'Dimensão', 'D2_00019')
        d2_00019.insert(2, 'Descrição da Dimensão', 'Verifica a Informação do valor patrimonial de bens imóveis')
        d2_00019.insert(3, 'OBS', 'Anexo I-AB da DCA')

        #################################################################################################################
        #D2_00020_Depreciação de Bens Imóveis
        depr_bens_imoveis = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1238102)")]
        depr_bens_imoveis['dimensao'] = 'D2_00020_Depreciação de Bens Imóveis'
        depr_bens_imoveis = depr_bens_imoveis.groupby('dimensao').agg({'VALOR': 'sum'})
        d2_00020_t = depr_bens_imoveis.reset_index()

        #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
        condicao = d2_00020_t['VALOR'] < 0  #como o sinal esta invertido da depreciação, para verificar precisei colocar menor que zero (para verificar que existe saldo)

        if condicao.any():
            resposta_d2_00020 = 'OK'
        else:
            resposta_d2_00020 = 'ERRO'

        d2_00020 = pd.DataFrame([resposta_d2_00020], columns=['Resposta'])
        d2_00020.insert(0, 'Dimensão', 'D2_00020')
        d2_00020.insert(2, 'Descrição da Dimensão', 'Verifica a Informação de depreciação acumulada de bens imóveis')
        d2_00020.insert(3, 'OBS', 'Anexo I-AB da DCA')

        #################################################################################################################
        # D2_00021_DIF_Entre Depreciação e Bens Imóveis
        dif_depr_bens_imov = pd.concat([bens_imoveis, depr_bens_imoveis])
        # Calcula a diferença
        diff = dif_depr_bens_imov['VALOR'].iloc[-2] - dif_depr_bens_imov['VALOR'].iloc[-1]
        # Cria uma linha no final com este cálculo da diferença usando pd.concat ao invés de append
        nova_linha = pd.DataFrame({'VALOR': [diff]})
        dif_depr_bens_imov = pd.concat([dif_depr_bens_imov, nova_linha], ignore_index=True)
        # Pega então somente esta linha final (da diferença)
        dif_depr_bens_imov = dif_depr_bens_imov.iloc[[len(dif_depr_bens_imov)-1]]
        dif_depr_bens_imov['dimensao'] = 'D2_00021_ DIF entre Depreciação e Bens Imóveis'
        d2_00021_t = dif_depr_bens_imov.reset_index()
        # Excluir a coluna Index e depois agrupar as contas novamente (só para vizualizar melhor a tabela)
        d2_00021_t.drop(columns=['index'], inplace=True)
        d2_00021_t = d2_00021_t.groupby('dimensao').sum().reset_index()

        #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
        condicao = d2_00021_t['VALOR'] > 0  

        if condicao.any():
            resposta_d2_00021 = 'OK'
        else:
            resposta_d2_00021 = 'ERRO'

        d2_00021 = pd.DataFrame([resposta_d2_00021], columns=['Resposta'])
        d2_00021.insert(0, 'Dimensão', 'D2_00021')
        d2_00021.insert(2, 'Descrição da Dimensão', 'Avalia se o valor dos bens imóveis é maior que sua depreciação acumulada')
        d2_00021.insert(3, 'OBS', 'Anexo I-AB da DCA')

        #################################################################################################################
        #D2_00028_Passivo Circulante Financeiro é menor ou igual ao valor de Passivo Circulante
        pass_circ = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(21)")]
        pass_circ_f = msc_base_e_ult_mes.loc[msc_base_e_ult_mes['CONTA'].str.match(r"^(21)") & (msc_base_e_ult_mes['IC2'] == "1")]
        pass_circ_f['VALOR'] = pass_circ_f['VALOR'] * -1

        d2_00028_t = pd.concat([pass_circ, pass_circ_f])
        d2_00028_t['dimensao'] = 'D2_00028_Passivo Circulante Financeiro <= ao Passivo Circulante'
        d2_00028_t = d2_00028_t.groupby('dimensao').agg({'VALOR': 'sum'})

        condicao = d2_00028_t['VALOR'] >= 0 

        if condicao.any():
            resposta_d2_00028 = 'OK'
        else:
            resposta_d2_00028 = 'ERRO'

        d2_00028 = pd.DataFrame([resposta_d2_00028], columns=['Resposta'])
        d2_00028.insert(0, 'Dimensão', 'D2_00028')
        d2_00028.insert(2, 'Descrição da Dimensão', 'Avalia se o valor de Passivo Circulante Financeiro é menor ou igual ao Passivo Circulante')
        d2_00028.insert(3, 'OBS', 'Anexo I-AB da DCA')


        #################################################################################################################
        #D2_00029
        vpd_juros = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(341)")]
        vpd_juros['dimensao'] = 'D2_00029_VPD de Juros'

        emprest = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(212|222)")]
        emprest['dimensao'] = 'D2_00029_Emprest. e Financ.'

        d2_00029_t = pd.concat([vpd_juros, emprest])
        d2_00029_t = d2_00029_t.groupby('dimensao').agg({'VALOR': 'sum'})

        # Verifica se 'emprest_cp' não está vazio
        if not emprest.empty:
            # Verifica se há algum saldo maior que zero em 'vpd_juros'
            # Substitua 'saldo' pela coluna que representa o saldo em 'vpd_juros'
            if (vpd_juros['VALOR'] > 0).any(): # verifica se existe empréstimos e caso exista, então verifique se existe alguma VPD de Juros
                resposta_d2_00029 = 'OK'
            else:
                resposta_d2_00029 = 'ERRO'
        else:
            resposta_d2_00029 = 'OK'

        d2_00029 = pd.DataFrame([resposta_d2_00029], columns=['Resposta'])
        d2_00029.insert(0, 'Dimensão', 'D2_00029')
        d2_00029.insert(2, 'Descrição da Dimensão', 'Verifica a existência de VPD de Juros e Encargos de Empréstimos e Financiamentos quando foram informados valores de Obrigações decorrentes de empréstimos e financiamentos')
        d2_00029.insert(3, 'OBS', 'Anexo I-AB e Anexo I-HI da DCA')

        #################################################################################################################
        #D2_00032_Reconhecimento de Ajuste de Dívida Ativa
        divida_ativa_cp_1 = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(11251|11261)")]
        divida_ativa_cp_1['dimensao'] = 'D2_00032_Dívida Ativa CP_EXTRA'
        divida_ativa_cp_1 = divida_ativa_cp_1.groupby('dimensao').agg({'VALOR': 'sum'})

        divida_ativa_cp_2 = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(11252|11262)")]
        divida_ativa_cp_2['dimensao'] = 'D2_00032_Dívida Ativa CP_INTRA'
        divida_ativa_cp_2 = divida_ativa_cp_2.groupby('dimensao').agg({'VALOR': 'sum'})

        divida_ativa_cp_3 = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(11253|11263)")]
        divida_ativa_cp_3['dimensao'] = 'D2_00032_Dívida Ativa CP_UNIAO'
        divida_ativa_cp_3 = divida_ativa_cp_3.groupby('dimensao').agg({'VALOR': 'sum'})

        divida_ativa_cp_4 = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(11254|11264)")]
        divida_ativa_cp_4['dimensao'] = 'D2_00032_Dívida Ativa CP_ESTADO'
        divida_ativa_cp_4 = divida_ativa_cp_4.groupby('dimensao').agg({'VALOR': 'sum'})

        divida_ativa_cp_5 = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(11255|11265)")]
        divida_ativa_cp_5['dimensao'] = 'D2_00032_Dívida Ativa CP_MUNICIPIO'
        divida_ativa_cp_5 = divida_ativa_cp_5.groupby('dimensao').agg({'VALOR': 'sum'})

        # Ajuste CP
        ajuste_divida_ativa_cp_1 = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1129104|1129105)")]
        ajuste_divida_ativa_cp_1['dimensao'] = 'D2_00032_Ajuste da Dívida Ativa CP_EXTRA'
        ajuste_divida_ativa_cp_1 = ajuste_divida_ativa_cp_1.groupby('dimensao').agg({'VALOR': 'sum'})

        ajuste_divida_ativa_cp_2 = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1129204|1129205)")]
        ajuste_divida_ativa_cp_2['dimensao'] = 'D2_00032_Ajuste da Dívida Ativa CP_INTRA'
        ajuste_divida_ativa_cp_2 = ajuste_divida_ativa_cp_2.groupby('dimensao').agg({'VALOR': 'sum'})

        ajuste_divida_ativa_cp_3 = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1129304|1129305)")]
        ajuste_divida_ativa_cp_3['dimensao'] = 'D2_00032_Ajuste da Dívida Ativa CP_UNIAO'
        ajuste_divida_ativa_cp_3 = ajuste_divida_ativa_cp_3.groupby('dimensao').agg({'VALOR': 'sum'})

        ajuste_divida_ativa_cp_4 = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1129404|1129405)")]
        ajuste_divida_ativa_cp_4['dimensao'] = 'D2_00032_Ajuste da Dívida Ativa CP_ESTADO'
        ajuste_divida_ativa_cp_4 = ajuste_divida_ativa_cp_4.groupby('dimensao').agg({'VALOR': 'sum'})

        ajuste_divida_ativa_cp_5 = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1129504|1129505)")]
        ajuste_divida_ativa_cp_5['dimensao'] = 'D2_00032_Ajuste da Dívida Ativa CP_MUNICIPIO'
        ajuste_divida_ativa_cp_5 = ajuste_divida_ativa_cp_5.groupby('dimensao').agg({'VALOR': 'sum'})


        # Divida LP
        divida_ativa_lp_1 = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1211104|1211105)")]
        divida_ativa_lp_1['dimensao'] = 'D2_00032_Dívida Ativa LP_EXTRA'
        divida_ativa_lp_1 = divida_ativa_lp_1.groupby('dimensao').agg({'VALOR': 'sum'})

        divida_ativa_lp_2 = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1211204|1211205)")]
        divida_ativa_lp_2['dimensao'] = 'D2_00032_Dívida Ativa LP_INTRA'
        divida_ativa_lp_2 = divida_ativa_lp_2.groupby('dimensao').agg({'VALOR': 'sum'})

        divida_ativa_lp_3 = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1211304|1211305)")]
        divida_ativa_lp_3['dimensao'] = 'D2_00032_Dívida Ativa LP_UNIAO'
        divida_ativa_lp_3 = divida_ativa_lp_3.groupby('dimensao').agg({'VALOR': 'sum'})

        divida_ativa_lp_4 = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1211404|1211405)")]
        divida_ativa_lp_4['dimensao'] = 'D2_00032_Dívida Ativa LP_ESTADO'
        divida_ativa_lp_4 = divida_ativa_lp_4.groupby('dimensao').agg({'VALOR': 'sum'})

        divida_ativa_lp_5 = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1211504|1211505)")]
        divida_ativa_lp_5['dimensao'] = 'D2_00032_Dívida Ativa LP_MUNICIPIO'
        divida_ativa_lp_5 = divida_ativa_lp_5.groupby('dimensao').agg({'VALOR': 'sum'})


        # AJuste LP
        ajuste_divida_ativa_lp_1 = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(121119904|121119905)")]
        ajuste_divida_ativa_lp_1['dimensao'] = 'D2_00032_Ajuste da Dívida Ativa LP_EXTRA'
        ajuste_divida_ativa_lp_1 = ajuste_divida_ativa_lp_1.groupby('dimensao').agg({'VALOR': 'sum'})

        ajuste_divida_ativa_lp_2 = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(121129904|121129905)")]
        ajuste_divida_ativa_lp_2['dimensao'] = 'D2_00032_Ajuste da Dívida Ativa LP_INTRA'
        ajuste_divida_ativa_lp_2 = ajuste_divida_ativa_lp_2.groupby('dimensao').agg({'VALOR': 'sum'})

        ajuste_divida_ativa_lp_3 = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(121139904|121139905)")]
        ajuste_divida_ativa_lp_3['dimensao'] = 'D2_00032_Ajuste da Dívida Ativa LP_UNIAO'
        ajuste_divida_ativa_lp_3 = ajuste_divida_ativa_lp_3.groupby('dimensao').agg({'VALOR': 'sum'})

        ajuste_divida_ativa_lp_4 = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(121149904|121149905)")]
        ajuste_divida_ativa_lp_4['dimensao'] = 'D2_00032_Ajuste da Dívida Ativa LP_ESTADO'
        ajuste_divida_ativa_lp_4 = ajuste_divida_ativa_lp_4.groupby('dimensao').agg({'VALOR': 'sum'})

        ajuste_divida_ativa_lp_5 = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(121159904|121159905)")]
        ajuste_divida_ativa_lp_5['dimensao'] = 'D2_00032_Ajuste da Dívida Ativa LP'
        ajuste_divida_ativa_lp_5 = ajuste_divida_ativa_lp_5.groupby('dimensao').agg({'VALOR': 'sum'})

        d2_00032_t = pd.concat([divida_ativa_cp_1, ajuste_divida_ativa_cp_1, divida_ativa_cp_2, ajuste_divida_ativa_cp_2, divida_ativa_cp_3, ajuste_divida_ativa_cp_3,
                                divida_ativa_cp_4, ajuste_divida_ativa_cp_4, divida_ativa_cp_5, ajuste_divida_ativa_cp_5,
                                divida_ativa_lp_1, ajuste_divida_ativa_lp_1, divida_ativa_lp_2, ajuste_divida_ativa_lp_2, divida_ativa_lp_3, ajuste_divida_ativa_lp_3,
                                divida_ativa_lp_4, ajuste_divida_ativa_lp_4, divida_ativa_lp_5, ajuste_divida_ativa_lp_5])
        d2_00032_t = d2_00032_t.reset_index()

        condicao6 = not ajuste_divida_ativa_lp_1.empty and (divida_ativa_lp_1['VALOR'] != 0).all()
        condicao7 = not ajuste_divida_ativa_lp_2.empty and (divida_ativa_lp_2['VALOR'] != 0).all()
        condicao8 = not ajuste_divida_ativa_lp_3.empty and (divida_ativa_lp_3['VALOR'] != 0).all()
        condicao9 = not ajuste_divida_ativa_lp_4.empty and (divida_ativa_lp_4['VALOR'] != 0).all()
        condicao10 = not ajuste_divida_ativa_lp_5.empty and (divida_ativa_lp_5['VALOR'] != 0).all()

        if condicao6 and condicao7 and condicao8 and condicao9:
            resposta_d2_00032 = 'OK'
        else:
            resposta_d2_00032 = 'ERRO'

        d2_00032 = pd.DataFrame([resposta_d2_00032], columns=['Resposta'])
        d2_00032.insert(0, 'Dimensão', 'D2_00032')
        d2_00032.insert(2, 'Descrição da Dimensão', 'Verifica a informação de Ajuste de Dívida Ativa (Tributária + Não Tributária)')
        d2_00032.insert(3, 'OBS', 'Anexo I-AB da DCA')


        #################################################################################################################
        #D2_00033_receita ou dedução de receita no Anexo I-C da DCA que não é de competência do ente (NÃO RECEITA)
        # Primeiro filtrando as contas de Receita
        msc_base_e_ult_mes_rec = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(621)")]
        # Depois Filtrando as Naturezas de Impostos
        msc_base_e_ult_mes_rec_impost = msc_base_e_ult_mes_rec[msc_base_e_ult_mes_rec['IC4'].str.match(r"^(111)")]
        # Depois pegando as que não são Ipostos Estaduais
        # Condição de NEgação (til) para pegar os Impostos que não São estaduais (coloca na regra os impostos estaduais e depois nego a sentença)
        d2_00033_t = msc_base_e_ult_mes_rec_impost[~msc_base_e_ult_mes_rec_impost['IC4'].str.match(r"^(11125|11130|11145)")] #11125 (pega IPVA final 51 e ITCMD final 52), 11130 (pega todo o IRRF) e 11145 pega todo o ICMS + Adicionais)

        condicao = d2_00033_t.empty

        if condicao:
            resposta_d2_00033 = 'OK'
        else:
            resposta_d2_00033 = 'ERRO'

        d2_00033 = pd.DataFrame([resposta_d2_00033], columns=['Resposta'])
        d2_00033.insert(0, 'Dimensão', 'D2_00033')
        d2_00033.insert(2, 'Descrição da Dimensão', 'Verifica a Informação de receitas que Não são de competência do ente')
        d2_00033.insert(3, 'OBS', 'Anexo I-C da DCA')

        #################################################################################################################
        #D2_00034
        msc_base_orig_e = msc_base_orig.query('TIPO_VALOR == "ending_balance"')
        # Pegando os 4 primeiros digitos
        msc_base_orig_e['q1'] = msc_base_orig_e['CONTA'].astype(str).str[:4]
        # Pegando o Quinto nível (nivel de análise)
        msc_base_orig_e['q2'] = msc_base_orig_e['CONTA'].astype(str).str[4:5]
        # Pegando o Sexto nível (pegar os zerados)
        msc_base_orig_e['q3'] = msc_base_orig_e['CONTA'].astype(str).str[5:6]

        d2_00034_t = msc_base_orig_e[(msc_base_orig_e['q2'] != '0') & (msc_base_orig_e['q2'] == '0')]

        condicao = d2_00034_t['VALOR'] >= 0 

        if condicao.all():
            resposta_d2_00034 = 'OK'
        else:
            resposta_d2_00034 = 'ERRO'

        d2_00034 = pd.DataFrame([resposta_d2_00034], columns=['Resposta'])
        d2_00034.insert(0, 'Dimensão', 'D2_00034')
        d2_00034.insert(2, 'Descrição da Dimensão', 'Verifica a existência de contas de quinto nível do PCASP com saldos negativos')
        d2_00034.insert(3, 'OBS', 'Anexo I-HI da DCA')

        # Removendo as colunas 'q1' 'q2' e 'q3'
        msc_base_orig_e.drop(columns=['q1', 'q2', 'q3'], inplace=True)

        #################################################################################################################
        #D2_00036_Créditos Tributários X VPA
        cred_trib = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1121|1211101|1211201|1211301|1211401|1211501)")]
        cred_trib['dimensao'] = 'D2_00036_Crédito Tributário'
        cred_trib = cred_trib.groupby('dimensao').agg({'VALOR': 'sum'})

        vpa_cred_trib = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(41)")]
        vpa_cred_trib['dimensao'] = 'D2_00036_VPA de Crédito Tributário'
        vpa_cred_trib = vpa_cred_trib.groupby('dimensao').agg({'VALOR': 'sum'})

        d2_00036_t = pd.concat([cred_trib, vpa_cred_trib])
        d2_00036_t = d2_00036_t.reset_index()

        #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
        condicao = d2_00036_t['VALOR'] != 0 #posso fazer diferente de zero, pois entende-se que todo Ente tem que ter VPA de Tributos e Contas a Receber

        if condicao.any():
            resposta_d2_00036 = 'OK'
        else:
            resposta_d2_00036 = 'ERRO'

        d2_00036 = pd.DataFrame([resposta_d2_00036], columns=['Resposta'])
        d2_00036.insert(0, 'Dimensão', 'D2_00036')
        d2_00036.insert(2, 'Descrição da Dimensão', 'Verifica a evidenciação de créditos tributários na DCA')
        d2_00036.insert(3, 'OBS', 'Anexo I-AB e Anexo I-HI da DCA')

        #################################################################################################################
        # D2_00037 (DÚVIDA)
        #P4.1.0.0.0.00.00
        d2_00037_t = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(41)")]
        d2_00037_t['dimensao'] = 'D2_00036_VPA de Crédito Tributário'
        d2_00037_t = d2_00037_t.groupby('dimensao').agg({'VALOR': 'sum'})

        condicao = d2_00037_t['VALOR'] >= 0 

        if condicao.all():
            resposta_d2_00037 = 'OK'
        else:
            resposta_d2_00037 = 'ERRO'

        d2_00037 = pd.DataFrame([resposta_d2_00037], columns=['Resposta'])
        d2_00037.insert(0, 'Dimensão', 'D2_00037')
        d2_00037.insert(2, 'Descrição da Dimensão', 'Verifica se houve registro de créditos tributários no período')
        d2_00037.insert(3, 'OBS', 'Anexo I-HI da DCA')

        #################################################################################################################
        ## D2_00038
        #D2_00038_Créditos Previdenciários a Receber
        cred_trib_rcb = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1136)")] # Créditos Previdenciários a Receber a CURTO PRAZO
        cred_trib_rcb['dimensao'] = 'D2_00038_Créditos Previdenciários a Receber'
        cred_trib_rcb = cred_trib_rcb.groupby('dimensao').agg({'VALOR': 'sum'})
        d2_00038_t = cred_trib_rcb.reset_index()

        #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
        condicao = d2_00038_t['VALOR'] > 0  

        if condicao.any():
            resposta_d2_00038 = 'OK'
        else:
            resposta_d2_00038 = 'ERRO'

        d2_00038 = pd.DataFrame([resposta_d2_00038], columns=['Resposta'])
        d2_00038.insert(0, 'Dimensão', 'D2_00038')
        d2_00038.insert(2, 'Descrição da Dimensão', 'Verifica a informação de créditos previdenciários')
        d2_00038.insert(3, 'OBS', 'Anexo I-AB da DCA')

        #################################################################################################################
        ## D2_00039
        prov_trab = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(2171|2271)")]
        prov_prev = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(2172|2272)")]
        prov_fiscal = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(2173|2273)")]
        prov_civeis = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(2174|2274)")]
        prov_outras = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(2179|2279)")]

        vpd_trab = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(3971)")]
        vpd_prev = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(3972)")]
        vpd_fiscal = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(3973)")]
        vpd_civeis = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(3974)")]
        vpd_outras = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(3979)")]

        trab = pd.concat([prov_trab, vpd_trab])
        trab = trab.groupby(['CONTA']).agg({'VALOR': 'sum'}).reset_index()
        prev = pd.concat([prov_prev, vpd_prev])
        prev = prev.groupby(['CONTA']).agg({'VALOR': 'sum'}).reset_index()
        fiscal = pd.concat([prov_fiscal, vpd_fiscal])
        fiscal = fiscal.groupby(['CONTA']).agg({'VALOR': 'sum'}).reset_index()
        civeis = pd.concat([prov_civeis, vpd_civeis])
        civeis = civeis.groupby(['CONTA']).agg({'VALOR': 'sum'}).reset_index()
        outras = pd.concat([prov_outras, vpd_outras])
        outras = outras.groupby(['CONTA']).agg({'VALOR': 'sum'}).reset_index()

        # Condições individuais
        condicao_1 = trab['CONTA'].str.startswith('3').any() # Verifica se existe alguma conta começando com '3' no Grupo de Trabalhista
        condicao_2 = ~trab['CONTA'].str.startswith('2').any() # Verifica se NÃO existe alguma conta começando com '2' no Grupo de Trabalhista
        condicao_3 = prev['CONTA'].str.startswith('3').any() # Verifica se existe alguma conta começando com '3' no Grupo de Previdenciário
        condicao_4 = ~prev['CONTA'].str.startswith('2').any() # Verifica se NÃO existe alguma conta começando com '2' no Grupo de Previdenciário
        condicao_5 = fiscal['CONTA'].str.startswith('3').any() # Verifica se existe alguma conta começando com '3' no Grupo de Fiscal
        condicao_6 = ~fiscal['CONTA'].str.startswith('2').any() # Verifica se NÃO existe alguma conta começando com '2' no Grupo de Fiscal
        condicao_7 = civeis['CONTA'].str.startswith('3').any() # Verifica se existe alguma conta começando com '3' no Grupo de Cíveis
        condicao_8 = ~civeis['CONTA'].str.startswith('2').any() # Verifica se NÃO existe alguma conta começando com '2' no Grupo de Cíveis
        condicao_9 = outras['CONTA'].str.startswith('3').any() # Verifica se existe alguma conta começando com '3' no Grupo de Outras
        condicao_10 = ~outras['CONTA'].str.startswith('2').any() # Verifica se NÃO existe alguma conta começando com '2' no Grupo de Outras

        # Combinando as condições em pares (por grupos do >> tipo_prov)
        par_1_e_2 = condicao_1 and condicao_2
        par_3_e_4 = condicao_3 and condicao_4
        par_5_e_6 = condicao_5 and condicao_6
        par_7_e_8 = condicao_7 and condicao_8
        par_9_e_10 = condicao_9 and condicao_10

        # Verificando se ambos os pares de condições são verdadeiros (se existir apenas um verdadeiro, ou seja >> Existe VPD sem PASSIVO = ERRO)
        resultado_final = par_1_e_2 and par_3_e_4 and par_5_e_6 and par_7_e_8 and par_9_e_10

        d2_00039_t = pd.concat([trab, prev, fiscal, civeis, outras])

        # Imprimindo o resultado
        if resultado_final.any():
            resposta_d2_00039 = 'ERRO'
        else:
            resposta_d2_00039 = 'OK'

        d2_00039 = pd.DataFrame([resposta_d2_00039], columns=['Resposta'])
        d2_00039.insert(0, 'Dimensão', 'D2_00039')
        d2_00039.insert(2, 'Descrição da Dimensão', 'Verifica se há valor de VPD de provisão, então deve haver o registro no passivo de curto ou de longo prazo')
        d2_00039.insert(3, 'OBS', 'Anexo I-AB e Anexo I-HI da DCA')


        #################################################################################################################
        # D2_00051
        #Verificar se o do total do Ajuste para perdas em Estoques é inferior ao total do saldo dos Estoques no Anexo I-AB da DCA.
        d2_00051_t = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(115)")]
        d2_00051_t['dimensao'] = 'D2_00051_Grupo de Estoques e Ajuste para Perdas'
        d2_00051_t = d2_00051_t.groupby('dimensao').agg({'VALOR': 'sum'}) # Se quiser abrir mais a INFO é Só agrupar por CONTA

        condicao = d2_00051_t['VALOR'] >= 0 

        if condicao.all():
            resposta_d2_00051 = 'OK'
        else:
            resposta_d2_00051 = 'ERRO'

        d2_00051 = pd.DataFrame([resposta_d2_00051], columns=['Resposta'])
        d2_00051.insert(0, 'Dimensão', 'D2_00051')
        d2_00051.insert(2, 'Descrição da Dimensão', 'Verificar se o do total do Ajuste para perdas em Estoques é inferior ao total do saldo dos Estoques')
        d2_00051.insert(3, 'OBS', ' Anexo I-AB da DCA')

        #################################################################################################################
        # D2_00052
        eq_patrim = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1221101|1221201|1221301|1221401|1221501)")]
        res_eq_patrim = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(392|492)")]

        d2_00052_t = pd.concat([eq_patrim, res_eq_patrim])
        d2_00052_t = d2_00052_t.groupby('CONTA').agg({'VALOR': 'sum'})

        # Verifica se 'res_eq_patrim' não está vazio
        if not res_eq_patrim.empty:
            # Verifica se há algum saldo maior que zero em 'eq_patrim'
            if (eq_patrim['VALOR'] > 0).any():
                resposta_d2_00052 = 'OK'
            else:
                resposta_d2_00052 = 'ERRO'
        else:
            resposta_d2_00052 = 'OK'

        d2_00052 = pd.DataFrame([resposta_d2_00052], columns=['Resposta'])
        d2_00052.insert(0, 'Dimensão', 'D2_00052')
        d2_00052.insert(2, 'Descrição da Dimensão', 'Verificar se o do total do Ajuste para perdas em Estoques é inferior ao total do saldo dos Estoques')
        d2_00052.insert(3, 'OBS', ' Anexo I-AB da DCA')

        #################################################################################################################
        # D2_00053
        d2_00053_t = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(115)")]
        d2_00053_t['dimensao'] = 'D2_00053_Grupo de Estoques e Ajuste para Perdas'
        d2_00053_t = d2_00053_t.groupby('dimensao').agg({'VALOR': 'sum'}) # Se quiser abrir mais a INFO é Só agrupar por CONTA

        condicao = d2_00053_t['VALOR'] >= 0 

        if condicao.all():
            resposta_d2_00053 = 'OK'
        else:
            resposta_d2_00053 = 'ERRO'

        d2_00053 = pd.DataFrame([resposta_d2_00053], columns=['Resposta'])
        d2_00053.insert(0, 'Dimensão', 'D2_00053')
        d2_00053.insert(2, 'Descrição da Dimensão', 'Verificar se o total do Ajuste para perdas em Estoques é inferior ao total do saldo dos Estoque')
        d2_00053.insert(3, 'OBS', 'MSC de Encerramento')

        #################################################################################################################
        # D2_00054
        eq_patrim = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1221101|1221201|1221301|1221401|1221501)")]
        res_eq_patrim = msc_base_b_ult_mes[msc_base_b_ult_mes['CONTA'].str.match(r"^(392|492)")]

        d2_00054_t = pd.concat([eq_patrim, res_eq_patrim])
        d2_00054_t = d2_00054_t.groupby('CONTA').agg({'VALOR': 'sum'})

        # Verifica se 'res_eq_patrim' não está vazio
        if not res_eq_patrim.empty:
            # Verifica se há algum saldo maior que zero em 'eq_patrim'
            if (eq_patrim['VALOR'] > 0).any():
                resposta_d2_00054 = 'OK'
            else:
                resposta_d2_00054 = 'ERRO'
        else:
            resposta_d2_00054 = 'OK'


        d2_00054 = pd.DataFrame([resposta_d2_00054], columns=['Resposta'])
        d2_00054.insert(0, 'Dimensão', 'D2_00054')
        d2_00054.insert(2, 'Descrição da Dimensão', 'Verificar se ente está registrando os investimentos permanentes, em conformidade com o PIPCP')
        d2_00054.insert(3, 'OBS', 'MSC de Encerramento')

        #################################################################################################################
        # D2_00055
        software = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1241|1248101)")]
        software['dimensao'] = 'D2_00055_Software'
        software = software.groupby(['dimensao'])['VALOR'].sum().reset_index()

        marca = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1242|1248102)")]
        marca['dimensao'] = 'D2_00055_Marca'
        marca = marca.groupby(['dimensao'])['VALOR'].sum().reset_index()

        d2_00055_t = pd.concat([software, marca])

        condicao = d2_00055_t['VALOR'] >= 0 

        if condicao.all():
            resposta_d2_00055 = 'OK'
        else:
            resposta_d2_00055 = 'ERRO'

        d2_00055 = pd.DataFrame([resposta_d2_00055], columns=['Resposta'])
        d2_00055.insert(0, 'Dimensão', 'D2_00055')
        d2_00055.insert(2, 'Descrição da Dimensão', 'Verificar (por grupo de ativos) se a amortização acumulada de ativos intangíveis é maior que o do ativo intangível')
        d2_00055.insert(3, 'OBS', 'MSC de Encerramento')


        #################################################################################################################
        # D2_00059
        # Para o Curto Prazo precisa fazer a associação por grupo entre o Contas a Receber e o seu Respectivo Ajuste para Perdas
        cred_trib_cp = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1121)")]
        cred_trib_cp['tipo'] = "CP_Credito Tributario"
        clientes_cp = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1122)")]
        clientes_cp['tipo'] = "CP_Clientes"
        emprest_e_financ_cp = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1124)")]
        emprest_e_financ_cp['tipo'] = "CP_Emprestimos e Financiamentos"
        divida_ativa_trib_cp = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1125)")]
        divida_ativa_trib_cp['tipo'] = "CP_Divida Ativa Tributaria"
        divida_ativa_nao_trib_cp = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1126)")]
        divida_ativa_nao_trib_cp['tipo'] = "CP_Divida Ativa Não Tributaria"

        ajuste_perda_cred_trib_cp = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^1129101|1129201|1129301|1129401|1129501")]
        ajuste_perda_cred_trib_cp['tipo'] = "CP_Credito Tributario"
        ajuste_perda_clientes_cp = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^1129102|1129202|1129302|1129402|1129502")]
        ajuste_perda_clientes_cp['tipo'] = "CP_Clientes"
        ajuste_perda_emprest_e_financ_cp = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^1129103|1129203|1129303|1129403|1129503")]
        ajuste_perda_emprest_e_financ_cp['tipo'] = "CP_Emprestimos e Financiamentos"
        ajuste_perda_divida_ativa_trib_cp = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^1129104|1129204|1129304|1129404|1129504")]
        ajuste_perda_divida_ativa_trib_cp['tipo'] = "CP_Divida Ativa Tributaria"
        ajuste_perda_divida_ativa_nao_trib_cp = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^1129105|1129205|1129305|1129405|1129505")]
        ajuste_perda_divida_ativa_nao_trib_cp['tipo'] = "CP_Divida Ativa Não Tributaria"

        # Já para o Longo Prazo, não precisa separar por Grupo de Contas a Receber e sim apenas pelo quinto nivel de consolidação
        cred_lp_extra = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(12111)")]
        cred_lp_extra['tipo'] = "LP_Credito EXTRA"
        cred_lp_intra = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(12112)")]
        cred_lp_intra['tipo'] = "LP_Credito INTRA"
        cred_lp_uniao = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(12113)")]
        cred_lp_uniao['tipo'] = "LP_Credito UNIAO"
        cred_lp_estado = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(12114)")]
        cred_lp_estado['tipo'] = "LP_Credito ESTADO"
        cred_lp_municipio = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(12115)")]
        cred_lp_municipio['tipo'] = "LP_Credito MUNICIPIO"

        d2_00059_t = pd.concat([cred_trib_cp,ajuste_perda_cred_trib_cp,clientes_cp, ajuste_perda_clientes_cp,emprest_e_financ_cp,ajuste_perda_emprest_e_financ_cp,
                                divida_ativa_trib_cp,ajuste_perda_divida_ativa_trib_cp,divida_ativa_nao_trib_cp,ajuste_perda_divida_ativa_nao_trib_cp,
                                cred_lp_extra,cred_lp_intra,cred_lp_uniao,cred_lp_estado,cred_lp_municipio])

        d2_00059_t = d2_00059_t.groupby(['tipo'])['VALOR'].sum().reset_index()


        #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
        # Verifica se existe algum valor negativo na coluna 'valor'
        condicao_negativa = (d2_00059_t['VALOR'] < 0).any()

        if condicao_negativa:
            # Se existe pelo menos um valor negativo, a resposta deve ser 'ERRO'
            resposta_d2_00059 = 'ERRO'
        else:
            # Se não existem valores negativos, a resposta é 'OK'
            resposta_d2_00059 = 'OK'

        d2_00059 = pd.DataFrame([resposta_d2_00059], columns=['Resposta'])
        d2_00059.insert(0, 'Dimensão', 'D2_00059')
        d2_00059.insert(2, 'Descrição da Dimensão', 'Verifica a relação entre o valor de ajuste para perdas dos "Créditos a curto e longo prazo" e suas contas principais. ')
        d2_00059.insert(3, 'OBS', 'MSC de Encerramento')


        #################################################################################################################
        # D2_00060
        # Para o Curto Prazo apenas pelo quinto nivel de consolidação
        outros_cred_cp_extra = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^11311|11321|11331|11341|11351|11361|11371|11381")]
        outros_cred_cp_extra['tipo'] = "CP_Outros Creditos EXTRA"
        outros_cred_cp_intra = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^11312|11322|11332|11342|11352|11362|11372|11382")]
        outros_cred_cp_intra['tipo'] = "CP_Outros Creditos INTRA"
        outros_cred_cp_uniao = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^11313|11323|11333|11343|11353|11363|11373|11383")]
        outros_cred_cp_uniao['tipo'] = "CP_Outros Creditos UNIAO"
        outros_cred_cp_estado = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^11314|11324|11334|11344|11354|11364|11374|11384")]
        outros_cred_cp_estado['tipo'] = "CP_Outros Creditos ESTADO"
        outros_cred_cp_municipio = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^11315|11325|11335|11345|11355|11365|11376|11385")]
        outros_cred_cp_municipio['tipo'] = "CP_Outros Creditos MUNICIPIO"

        # Ajustes do CP
        ajuste_outros_cred_cp_extra = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(11391)")]
        ajuste_outros_cred_cp_extra['tipo'] = "CP_Outros Creditos EXTRA"
        ajuste_outros_cred_cp_intra = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(11392)")]
        ajuste_outros_cred_cp_intra['tipo'] = "CP_Outros Creditos INTRA"
        ajuste_outros_cred_cp_uniao = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(11393)")]
        ajuste_outros_cred_cp_uniao['tipo'] = "CP_Outros Creditos UNIAO"
        ajuste_outros_cred_cp_estado = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(11394)")]
        ajuste_outros_cred_cp_estado['tipo'] = "CP_Outros Creditos ESTADO"
        ajuste_outros_cred_cp_municipio = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(11395)")]
        ajuste_outros_cred_cp_municipio['tipo'] = "CP_Outros Creditos MUNICIPIO"

        # Já para o Longo Prazo, não precisa separar por Grupo de Contas a Receber e sim apenas pelo quinto nivel de consolidação
        outros_cred_lp_extra = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(12121)")]
        outros_cred_lp_extra['tipo'] = "LP_Outros Creditos EXTRA"
        outros_cred_lp_intra = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(12122)")]
        outros_cred_lp_intra['tipo'] = "LP_Outros Creditos INTRA"
        outros_cred_lp_uniao = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(12123)")]
        outros_cred_lp_uniao['tipo'] = "LP_Outros Creditos UNIAO"
        outros_cred_lp_estado = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(12124)")]
        outros_cred_lp_estado['tipo'] = "LP_Outros Creditos ESTADO"
        outros_cred_lp_municipio = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(12125)")]
        outros_cred_lp_municipio['tipo'] = "LP_Outros Creditos MUNICIPIO"

        d2_00060_t = pd.concat([outros_cred_cp_extra, outros_cred_cp_intra, outros_cred_cp_uniao, outros_cred_cp_estado, outros_cred_cp_municipio,
        ajuste_outros_cred_cp_extra,ajuste_outros_cred_cp_intra,ajuste_outros_cred_cp_uniao,ajuste_outros_cred_cp_estado,ajuste_outros_cred_cp_municipio,
                                outros_cred_lp_extra,outros_cred_lp_intra,outros_cred_lp_uniao,outros_cred_lp_estado,outros_cred_lp_municipio])

        d2_00060_t = d2_00060_t.groupby(['tipo'])['VALOR'].sum().reset_index()


        #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
        # Verifica se existe algum valor negativo na coluna 'valor'
        condicao_negativa = (d2_00060_t['VALOR'] < 0).any()

        if condicao_negativa:
            # Se existe pelo menos um valor negativo, a resposta deve ser 'ERRO'
            resposta_d2_00060 = 'ERRO'
        else:
            # Se não existem valores negativos, a resposta é 'OK'
            resposta_d2_00060 = 'OK'

        d2_00060 = pd.DataFrame([resposta_d2_00060], columns=['Resposta'])
        d2_00060.insert(0, 'Dimensão', 'D2_00060')
        d2_00060.insert(2, 'Descrição da Dimensão', 'Verifica a relação entre o valor de ajuste para perdas dos "Demais créditos e valores a curto e longo prazo" e suas contas principais"')
        d2_00060.insert(3, 'OBS', 'MSC de Encerramento')


        #################################################################################################################
        # D2_00061
        d2_00061_t = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(4522)")]
        d2_00061_t = d2_00061_t.groupby(['CONTA'])['VALOR'].sum().reset_index()

        condicao = d2_00061_t['VALOR'] >= 0 

        if condicao.all():
            resposta_d2_00061 = 'OK'
        else:
            resposta_d2_00061 = 'ERRO'

        d2_00061 = pd.DataFrame([resposta_d2_00061], columns=['Resposta'])
        d2_00061.insert(0, 'Dimensão', 'D2_00061')
        d2_00061.insert(2, 'Descrição da Dimensão', 'Verificar se foi informada Variação Patrimonial Aumentativa com o FUNDEB')
        d2_00061.insert(3, 'OBS', 'DCA - Anexo I-HI')

        #################################################################################################################
        # D2_00066
        software = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1241|1248101)")]
        software['dimensao'] = 'D2_00055_Software'
        software = software.groupby(['dimensao'])['VALOR'].sum().reset_index()

        marca = msc_base_e_ult_mes[msc_base_e_ult_mes['CONTA'].str.match(r"^(1242|1248102)")]
        marca['dimensao'] = 'D2_00055_Marca'
        marca = marca.groupby(['dimensao'])['VALOR'].sum().reset_index()

        d2_00066_t = pd.concat([software, marca])

        condicao = d2_00066_t['VALOR'] >= 0 

        if condicao.all():
            resposta_d2_00066 = 'OK'
        else:
            resposta_d2_00066 = 'ERRO'

        d2_00066 = pd.DataFrame([resposta_d2_00066], columns=['Resposta'])
        d2_00066.insert(0, 'Dimensão', 'D2_00066')
        d2_00066.insert(2, 'Descrição da Dimensão', 'Verifica, na DCA, se o valor de amortização acumulada é inferior ao valor do ativo intangível. A avaliação é feita por tipo de ativo intangível.')
        d2_00066.insert(3, 'OBS', 'DCA - Anexo I-AB')

        #################################################################################################################
        # D2_00067
        dif_depr_bens_mov = pd.concat([bens_moveis, depr_bens_moveis])
        # Calcula a diferença
        diff = dif_depr_bens_mov['VALOR'].iloc[-2] - dif_depr_bens_mov['VALOR'].iloc[-1]
        # Cria uma linha no final com este cálculo da diferença usando pd.concat ao invés de append
        nova_linha = pd.DataFrame({'VALOR': [diff]})
        dif_depr_bens_mov = pd.concat([dif_depr_bens_mov, nova_linha], ignore_index=True)
        # Pega então somente esta linha final (da diferença)
        dif_depr_bens_mov = dif_depr_bens_mov.iloc[[len(dif_depr_bens_mov)-1]]
        dif_depr_bens_mov['dimensao'] = 'D2_00067_DIF Depreciação e Bens Móveis'
        d2_00067_t = dif_depr_bens_mov.reset_index()
        # Excluir a coluna Index e depois agrupar as contas novamente (só para vizualizar melhor a tabela)
        d2_00067_t.drop(columns=['index'], inplace=True)
        d2_00067_t = d2_00067_t.groupby('dimensao').sum().reset_index()

        #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
        condicao = d2_00067_t['VALOR'] > 0  

        if condicao.any():
            resposta_d2_00067 = 'OK'
        else:
            resposta_d2_00067 = 'ERRO'

        d2_00067 = pd.DataFrame([resposta_d2_00067], columns=['Resposta'])
        d2_00067.insert(0, 'Dimensão', 'D2_00067')
        d2_00067.insert(2, 'Descrição da Dimensão', 'Verifica se os valores de depreciação de bens móveis são inferiores ao valor total de bens móveis')
        d2_00067.insert(3, 'OBS', 'MSC de Encerramento')

        #################################################################################################################
        # D2_00068
        dif_depr_bens_imov = pd.concat([bens_imoveis, depr_bens_imoveis])
        # Calcula a diferença
        diff = dif_depr_bens_imov['VALOR'].iloc[-2] - dif_depr_bens_imov['VALOR'].iloc[-1]
        # Cria uma linha no final com este cálculo da diferença usando pd.concat ao invés de append
        nova_linha = pd.DataFrame({'VALOR': [diff]})
        dif_depr_bens_imov = pd.concat([dif_depr_bens_imov, nova_linha], ignore_index=True)
        # Pega então somente esta linha final (da diferença)
        dif_depr_bens_imov = dif_depr_bens_imov.iloc[[len(dif_depr_bens_imov)-1]]
        dif_depr_bens_imov['dimensao'] = 'D2_00068_ DIF entre Depreciação e Bens Imóveis'
        d2_00068_t = dif_depr_bens_imov.reset_index()
        # Excluir a coluna Index e depois agrupar as contas novamente (só para vizualizar melhor a tabela)
        d2_00068_t.drop(columns=['index'], inplace=True)
        d2_00068_t = d2_00068_t.groupby('dimensao').sum().reset_index()

        #FAZER A CONDIÇÃO PARA DAR A RESPOSTA EM UMA LINHA (APÓS VERIFICAÇÃO)
        condicao = d2_00068_t['VALOR'] > 0  

        if condicao.any():
            resposta_d2_00068 = 'OK'
        else:
            resposta_d2_00068 = 'ERRO'

        d2_00068 = pd.DataFrame([resposta_d2_00068], columns=['Resposta'])
        d2_00068.insert(0, 'Dimensão', 'D2_00068')
        d2_00068.insert(2, 'Descrição da Dimensão', 'Verifica se os valores depreciação de bens imóveis são inferiores ao valor total de bens imóveis')
        d2_00068.insert(3, 'OBS', 'MSC de Encerramento')

        #################################################################################################################
        # D2_00076

        # Verifica se existem registros para os meses de referência 1 e 12
        verificar_saldos = (
            (msc_base['CONTA'].str.startswith("4425202")) | 
            (msc_base['CONTA'].str.startswith("1121271")) |
            (msc_base['CONTA'].str.startswith("1211206")) | 
            (msc_base['CONTA'].str.startswith("1136202")) 
        )

        # Filtra o DataFrame com base na condição e verifica se o resultado é vazio
        if not msc_base[verificar_saldos].empty:
            # Filtrando usando indexação booleana
            filtro_vpa = msc_base[
                (msc_base['TIPO_VALOR'] == "beginning_balance") & 
                (msc_base['mes'] == "01") &
                (msc_base['CONTA'].str.startswith("4425202"))
            ]

            filtro_vpa = filtro_vpa.groupby(['CONTA'])['VALOR'].sum().reset_index()

            filtro_ativo = msc_base[
                (msc_base['TIPO_VALOR'] == "ending_balance") & 
                (msc_base['mes'] == {ultimo_mes}) &
                (msc_base['CONTA'].str.startswith("1121271") |
                msc_base['CONTA'].str.startswith("1211206") |
                msc_base['CONTA'].str.startswith("1136202"))
            ]

            filtro_ativo = filtro_ativo.groupby(['CONTA'])['VALOR'].sum().reset_index()


            # Fazendo o merge dos DataFrames baseado em 'conta_contabil'
            d2_00076_t = pd.merge(filtro_vpa, filtro_ativo, on='CONTA', suffixes=('_inicio', '_fim'))

            # Calculando a diferença de 'valor' para cada 'conta_contabil'
            d2_00076_t['DIF'] = d2_00076_t['VALOR_fim'] - d2_00076_t['VALOR_inicio']

            # Condição para dar a resposta
            condicao = d2_00076_t['DIF'] == 0 

            if condicao.any():
                resposta_d2_00076 = 'ERRO'
            else:
                resposta_d2_00076 = 'OK'

            d2_00076 = pd.DataFrame([resposta_d2_00076], columns=['Resposta'])
            d2_00076.insert(0, 'Dimensão', 'D2_00076')
            d2_00076.insert(2, 'Descrição da Dimensão', 'Verifica se o saldo inicial da conta 4.4.2.5.2.02.XX na MSC de encerramento é maior 0, o saldo das contas (1.1.2.1.2.71.XX + 1.2.1.1.2.06.XX + 1.1.3.6.2.02.XX ) deve ser maior que 0')
            d2_00076.insert(3, 'OBS', 'MSC de Encerramento')


        else:
            # Cria um DataFrame com a informação de vazio se não existirem registros para ambos os meses
            d2_00076 = pd.DataFrame({
                'Dimensão': ['D2_00076'],
                'Resposta': ['VAZIO - Nenhum Saldo para as contas analisadas'],
                'Descrição da Dimensão': ['Verifica se o saldo inicial da conta 4.4.2.5.2.02.XX na MSC de encerramento é maior 0, o saldo das contas (1.1.2.1.2.71.XX + 1.2.1.1.2.06.XX + 1.1.3.6.2.02.XX ) deve ser maior que 0'],
                'OBS': ['MSC de Encerramento'],
            })

        #################################################################################################################
        # D2_00077
        # Seu código continua aqui para quando existem registros para ambos os meses
        filtro_1 = msc_base[
            (msc_base['TIPO_VALOR'] == "beginning_balance") & 
            (msc_base['mes'] == "01") &
            (msc_base['CONTA'].str.startswith("227") | 
                msc_base['CONTA'].str.startswith("228"))
        ]
        filtro_1 = filtro_1.groupby(['CONTA'])['VALOR'].sum().reset_index()

        filtro_2 = msc_base[
            (msc_base['TIPO_VALOR'] == "ending_balance") & 
            (msc_base['mes'] == {ultimo_mes}) &
            (msc_base['CONTA'].str.startswith("227") | 
                msc_base['CONTA'].str.startswith("228"))
        ]
        filtro_2 = filtro_2.groupby(['CONTA'])['VALOR'].sum().reset_index()

        # Fazendo o merge dos DataFrames baseado em 'conta_contabil'
        d2_00077_t = pd.merge(filtro_1, filtro_2, on='CONTA', suffixes=('_inicio', '_fim'))

        # Calculando a diferença de 'valor' para cada 'conta_contabil'
        d2_00077_t['DIF'] = d2_00077_t['VALOR_fim'] - d2_00077_t['VALOR_inicio']

        # Condição para dar a resposta
        condicao = d2_00077_t['DIF'] == 0 

        if condicao.any():
            resposta_d2_00077 = 'ERRO'
        else:
            resposta_d2_00077 = 'OK'

        d2_00077 = pd.DataFrame([resposta_d2_00077], columns=['Resposta'])
        d2_00077.insert(0, 'Dimensão', 'D2_00077')
        d2_00077.insert(2, 'Descrição da Dimensão', 'Comparativo do saldo das contas começadas por 227 e 228')
        d2_00077.insert(3, 'OBS', 'msc de janeiro (saldo inicial) com a msc de dezembro (saldo final)')


        #################################################################################################################
        # D2_00079
        # Seu código continua aqui para quando existem registros para ambos os meses
        filtro_1 = msc_base[
            (msc_base['TIPO_VALOR'] == "beginning_balance") & 
            (msc_base['mes'] == "01") &
            (msc_base['CONTA'].str.startswith("119"))
        ]
        filtro_1 = filtro_1.groupby(['CONTA'])['VALOR'].sum().reset_index()

        filtro_2 = msc_base[
            (msc_base['TIPO_VALOR'] == "ending_balance") & 
            (msc_base['mes'] == {ultimo_mes}) &
            (msc_base['CONTA'].str.startswith("119"))
        ]
        filtro_2 = filtro_2.groupby(['CONTA'])['VALOR'].sum().reset_index()

        # Fazendo o merge dos DataFrames baseado em 'conta_contabil'
        d2_00079_t = pd.merge(filtro_1, filtro_2, on='CONTA', suffixes=('_inicio', '_fim'))

        # Calculando a diferença de 'valor' para cada 'conta_contabil'
        d2_00079_t['DIF'] = d2_00079_t['VALOR_fim'] - d2_00079_t['VALOR_inicio']

        # Condição para dar a resposta
        condicao = d2_00079_t['DIF'] == 0 

        if condicao.any():
            resposta_d2_00079 = 'ERRO'
        else:
            resposta_d2_00079 = 'OK'

        d2_00079 = pd.DataFrame([resposta_d2_00079], columns=['Resposta'])
        d2_00079.insert(0, 'Dimensão', 'D2_00079')
        d2_00079.insert(2, 'Descrição da Dimensão', 'Comparativo do saldo das contas começadas por 119')
        d2_00079.insert(3, 'OBS', 'msc de janeiro (saldo inicial) com a msc de dezembro (saldo final)')

        #################################################################################################################
        # D2_00080
        filtro_1 = msc_base_e[
        (msc_base_e['CONTA'].str.startswith("1156"))
        ]

        d2_00080_t = filtro_1.groupby(['CONTA'])['VALOR'].sum().reset_index()

        condicao = d2_00080_t['VALOR'] > 0  

        ## Aqui Tem que ser o ANY (e não o ALL), porque precisa apenas que tenha uma conta do grupo 1156 com saldo nos 12 meses
        if condicao.any():
            resposta_d2_00080 = 'OK'
        else:
            resposta_d2_00080 = 'ERRO'

        d2_00080 = pd.DataFrame([resposta_d2_00080], columns=['Resposta'])
        d2_00080.insert(0, 'Dimensão', 'D2_00080')
        d2_00080.insert(2, 'Descrição da Dimensão', 'Avaliação do saldo das contas contábeis começadas por 1156')
        d2_00080.insert(3, 'OBS', 'Em todas as MSCs')

        #################################################################################################################
        # D2_00081
        filtro_2 = msc_base_e[
        ((msc_base_e['CONTA'].str.startswith("211110102")) | (msc_base_e['CONTA'].str.startswith("211110103")))
        ]

        d2_00081_t = filtro_2.groupby(['CONTA'])['VALOR'].sum().reset_index()

        condicao = d2_00081_t['VALOR'] > 0  

        ## Aqui Tem que ser o ANY (e não o ALL), porque precisa apenas que tenha uma conta do grupo 1156 com saldo nos 12 meses
        if condicao.any():
            resposta_d2_00081 = 'OK'
        else:
            resposta_d2_00081 = 'ERRO'

        d2_00081 = pd.DataFrame([resposta_d2_00081], columns=['Resposta'])
        d2_00081.insert(0, 'Dimensão', 'D2_00081')
        d2_00081.insert(2, 'Descrição da Dimensão', 'Avalia a existência de movimento credor nas contas 2.1.1.1.1.01.02 e 2.1.1.1.1.01.03')
        d2_00081.insert(3, 'OBS', 'MSCs mensais (janeiro a dezembro)')

        #################################################################################################################
        # D2_00082
        filtro_3 = msc_base_p[
            (msc_base_p['NATUREZA_VALOR'] == "C") &
            (msc_base_p['CONTA'].str.startswith("1238101") | 
            msc_base_p['CONTA'].str.startswith("1238103") | 
            msc_base_p['CONTA'].str.startswith("1238105"))
        ]

        d2_00082_t = filtro_3.groupby(['CONTA'])['VALOR'].sum().reset_index()

        condicao = d2_00082_t['VALOR'] > 0  

        if condicao.any():
            resposta_d2_00082 = 'OK'
        else:
            resposta_d2_00082 = 'ERRO'

        d2_00082 = pd.DataFrame([resposta_d2_00082], columns=['Resposta'])
        d2_00082.insert(0, 'Dimensão', 'D2_00082')
        d2_00082.insert(2, 'Descrição da Dimensão', 'Avalia a existência de movimento credor nas contas 1.2.3.8.1.01.XX, 1.2.3.8.1.03.XX e 1.2.3.8.1.05.XX')
        d2_00082.insert(3, 'OBS', 'MSCs mensais (janeiro a dezembro)')

        #################################################################################################################
        # D2_00083
        # Seu código original para filtrar e agrupar os dados
        filtro_1 = msc_base_e[
            (msc_base_e['CONTA'].str.startswith("721"))
        ]
        filtro_1 = filtro_1.groupby(['TIPO_VALOR'])['VALOR'].sum().reset_index()

        filtro_2 = msc_base_e[
            (msc_base_e['CONTA'].str.startswith("821"))
        ]
        filtro_2 = filtro_2.groupby(['TIPO_VALOR'])['VALOR'].sum().reset_index()

        # Fazendo o merge dos DataFrames baseado em 'conta_contabil'
        d2_00083_t = pd.merge(filtro_1, filtro_2, on='TIPO_VALOR', suffixes=('_7', '_8'))

        # Calculando a diferença de 'valor' para cada 'conta_contabil'
        d2_00083_t['DIF'] = d2_00083_t['VALOR_7'] - d2_00083_t['VALOR_8']

        # Defina uma tolerância absoluta. Por exemplo, 0.00001 para considerar diferenças menores que isso como zero.
        tolerancia = 0.001

        # Use numpy.isclose para comparar cada valor com 0, considerando a tolerância definida
        condicao = np.isclose(d2_00083_t['DIF'], 0, atol=tolerancia)

        if condicao.all():
            resposta_d2_00083 = 'OK'
        else:
            resposta_d2_00083 = 'ERRO'

        d2_00083 = pd.DataFrame([resposta_d2_00083], columns=['Resposta'])
        d2_00083.insert(0, 'Dimensão', 'D2_00083')
        d2_00083.insert(2, 'Descrição da Dimensão', 'Avalia o saldo final das contas contábeis começadas por 721 e 821 são iguais')
        d2_00083.insert(3, 'OBS', 'Em todas as MSCs')

        #################################################################################################################

        #################################################################################################################

        # Exibindo os resultados das análises
        d2_m = pd.concat([d2_00002, d2_00003, d2_00004, d2_00005, d2_00006, d2_00007, d2_00008, d2_00010, d2_00011, d2_00012,
                        d2_00013, d2_00014, d2_00015, d2_00016, d2_00017, d2_00018, d2_00019, d2_00020, d2_00021, d2_00028,
                        d2_00029, d2_00032, d2_00033, d2_00034, d2_00036, d2_00037, d2_00038, d2_00039, d2_00051, d2_00052,
                        d2_00053, d2_00054, d2_00055, d2_00059, d2_00060, d2_00061, d2_00066, d2_00067, d2_00068, d2_00076,
                        d2_00077, d2_00079, d2_00080, d2_00081, d2_00082, d2_00083])



C:\Users\mjandussi\AppData\Local\Temp\ipykernel_15832\1205507155.py:930: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  d2_00007_t = msc_base_e_ult_mes[dps_custeio]


In [10]:
d1_m_erros = d1_m.query('Resposta == "ERRO"')
d2_m_erros = d2_m.query('Resposta == "ERRO"')
d_erros = pd.concat([d1_m_erros, d2_m_erros])
d_erros = d_erros.reset_index(drop=True)

d_erros

,Dimensão,Resposta,Descrição da Dimensão,OBS
0,D1_00020,ERRO,Verifica a quantidade de matrizes com SI <> do SF da MSC do mês anterior,Cada MSC vale 1/12 - Erros: 1
1,D1_00025,ERRO,"Verifica a quantidade de matrizes com contas dos grupos 2111, 2112, 2113,2114, 2121, 2122, 2123, 2124, 2125,2126, 213, 214, 215, 221, 222, 223 cujo saldo final está com natureza diferente da padrão do PCASP Estendido",Cada MSC vale 1/13 - Erros: 1
2,D2_00032,ERRO,Verifica a informação de Ajuste de Dívida Ativa (Tributária + Não Tributária),Anexo I-AB da DCA


In [11]:
d1_00025_t

,CONTA,mes,NATUREZA DO SALDO,NATUREZA_VALOR,VALOR,TÍTULO.1,STATUS,CLASSE,GRUPO,SUBGRUPO,TÍTULO,SUBTÍTULO,ITEM,SUBITEM,FUNÇÃO,NÍVEL DETALHADO,INDICADOR DO SUPERÁVIT FINANCEIRO,ID IC,Informação Complementar
527,215150100,02,Credora,CD,"-31,767,207.17",REPARTIÇÃO DA COTA DE ICMS REPARTIÇÃO DA COTA DE ICMS,AtivaAtiva,22,11,55,11,55,0101,0000,Compreende/Registra o valor da cota de ICMS arrecadado devida aos municípios.Compreende/Registra o valor da cota de ICMS arrecadado devida aos municípios.,ÚltimoÚltimo,Financeiro/PermanenteFinanceiro/Permanente,8,PO - FP - FR - COPO - FP - FR - CO
536,215150200,02,Credora,CD,"-536,548.66",REPARTIÇÃO DA COTA DE IPVAREPARTIÇÃO DA COTA DE IPVA,AtivaAtiva,22,11,55,11,55,0202,0000,Compreende/Registra o valor da cota de IPVA arrecadado devida aos municípios.Compreende/Registra o valor da cota de IPVA arrecadado devida aos municípios.,ÚltimoÚltimo,Financeiro/PermanenteFinanceiro/Permanente,8,PO - FP - FR - COPO - FP - FR - CO


In [12]:
d1_00020_t

,CONTA,mes,TIPO_VALOR,VALOR,diferenca_valor
22536,522190209,04,beginning_balance,"52,641,602.94","-44,683,679.44"


In [13]:
d1_00027_t

,CONTA


In [14]:
d2_00032_t

,dimensao,VALOR
0,D2_00032_Dívida Ativa CP_EXTRA,"1,047,195,278.90"
1,D2_00032_Ajuste da Dívida Ativa CP_EXTRA,"-1,022,721,368.41"
2,D2_00032_Dívida Ativa LP_EXTRA,"172,432,971,298.58"
3,D2_00032_Ajuste da Dívida Ativa LP_EXTRA,"-168,310,568,024.14"
4,D2_00032_Dívida Ativa LP_INTRA,"201,976.53"
5,D2_00032_Dívida Ativa LP_MUNICIPIO,"388,956.23"


In [15]:
##########@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@############################################################    
##########@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@############################################################
##########@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@############################################################
##########@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@   Montando os HTMLs   @@@@@@@@@@@@@@@@@@##################################

html0 = d_erros.to_html(index=False)

html1 = d1_m.to_html(index=False)

html2 = d2_m.to_html(index=False)

In [ ]:
#########@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@############
#########@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@   Configurações de Email   @@@@@####

# Configurações
tipo_email = "outlook"  # ou "gmail"
tipo_destinatario = "pessoal"  # ou "grupo"

# Definir destinatários
if tipo_destinatario == "grupo":
    destinatarios = 'mjandussi@fazenda.rj.gov.br; cbborba@fazenda.rj.gov.br; Daniela.Alves@fazenda.rj.gov.br; Nathalia.Souza@fazenda.rj.gov.br; Gustavo.Mello@fazenda.rj.gov.br'
else:  # pessoal
    destinatarios = 'mjandussi@fazenda.rj.gov.br'

#########@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@   Envio do email   @@@@@############

if tipo_email == "outlook":

    import win32com.client as win32

    ######## Começar a montar o email
    outlook = win32.Dispatch('outlook.application')

    ########## Enviando o email
    mail = outlook.CreateItem(0)
    mail.To = destinatarios
    mail.Subject = f'Analise MSC do Ranking Siconfi - Mês {ultimo_mes} / {ano}'
    mail.HTMLBody = f"""
                    <p>Prezados,</p>
                    <p><b>Relação de Erros </b></p>
                    <p><b>Relação de Erros -  Análise Acumulada MSC até o mês {ultimo_mes} / {ano}</b></p>
                    <p>{html0}</p>
                    <p><b>OBS: as análises da Dimensão D2 são apenas antecipações da DCA feitas pela MSC</b></p>
                    <p>-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------</p>
                    <p></p>
                    <p><b>ITEM 1</b></p>
                    <p><b>Dimensão D1 -  Análise Acumulada MSC até o mês {ultimo_mes} / {ano}</b></p>
                    <p>{html1}</p>
                    <p></p>
                    <p>-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------</p>
                    <p><b>ITEM 3</b></p>
                    <p><b>Dimensão D2 (análises antecipadas da DCA feitas pela MSC) - Análise Acumulada MSC até o mês {ultimo_mes} / {ano}</b></p>
                    <p>{html2}</p>
                    <p></p>
                    <p>-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------</p>
                    """

    mail.Send()
    print('Email enviado')

##############################################################################################################################################################

else:

    import smtplib
    import email.message

    def enviar_email():
    
      corpo_email = f"""
                <p>Prezados,</p>
                    <p><b>Relação de Erros </b></p>
                    <p><b>Relação de Erros -  Análise Acumulada MSC até o mês {ultimo_mes} / {ano}</b></p>
                    <p>{html0}</p>
                    <p><b>OBS: as análises da Dimensão D2 são apenas antecipações da DCA feitas pela MSC</b></p>
                    <p>-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------</p>
                    <p></p>
                    <p><b>ITEM 1</b></p>
                    <p><b>Dimensão D1 -  Análise Acumulada MSC até o mês {ultimo_mes} / {ano}</b></p>
                    <p>{html1}</p>
                    <p></p>
                    <p>-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------</p>
                    <p><b>ITEM 3</b></p>
                    <p><b>Dimensão D2 (análises antecipadas da DCA feitas pela MSC) - Análise Acumulada MSC até o mês {ultimo_mes} / {ano}</b></p>
                    <p>{html2}</p>
                    <p></p>
                    <p>-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------</p>
                    """

      msg = email.message.Message()
      msg['Subject'] = f'Analise MSC do Ranking Siconfi - Mês {ultimo_mes} / {ano}'
      msg['From'] = 'mjandussi@gmail.com'
      msg['To'] = destinatarios
      password = 'lgdqjxacnesvnovm' 
      msg.add_header('Content-Type', 'text/html')
      msg.set_payload(corpo_email)
      s = smtplib.SMTP('smtp.gmail.com: 587')
      s.starttls()
      # Login Credentials for sending the mail
      s.login(msg['From'], password)
      s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('iso-8859-1')) #utf-8
      print('Email enviado')

    enviar_email()

# Outros Destinatários da SUBCONT

#Nathalia Ribeiro De Souza <Nathalia.Souza@fazenda.rj.gov.br>; Gustavo Moreira De Mello <Gustavo.Mello@fazenda.rj.gov.br>; Yasmim Da Costa Monteiro <ymonteiro@fazenda.rj.gov.br>; Bruno Campos Pereira <Bruno.Pereira@fazenda.rj.gov.br>; David Lopes De Souza <dldsouza@fazenda.rj.gov.br>; Fernanda Rodrigues De Oliveira Marques <Fernanda.Marques@fazenda.rj.gov.br>; Ronald Marcio Guedes Rodrigues <rrodrigues@fazenda.rj.gov.br>; Renato Ferreira Costa <rfcosta@fazenda.rj.gov.br>; Marcelo Vinicius De Macedo Bezerra <mvbezerra@fazenda.rj.gov.br>; Yago Barros Barbosa <Yago.Barbosa@fazenda.rj.gov.br>